In [130]:
"""
CHAINE DE MACHINE LEARNING
Projet: Prediction du risque d'infarctus
Dataset: dataset_infarctus.xlsx
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
import warnings

warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

In [131]:
"""
1. CHARGEMENT DES DONNEES
"""
print("\n" + "=" * 80)
print("1. CHARGEMENT ET EXPLORATION INITIALE")
print("=" * 80)

# Charger le dataset
df = pd.read_excel('data/dataset_infarctus.xlsx')

print(f"\nDimensions du dataset: {df.shape[0]} patients x {df.shape[1]} variables")
print(f"\nVariables disponibles:")
for i, col in enumerate(df.columns, 1):
    print(f"   {i:2d}. {col}")



1. CHARGEMENT ET EXPLORATION INITIALE

Dimensions du dataset: 1000 patients x 21 variables

Variables disponibles:
    1. age
    2. sexe
    3. imc
    4. hypertension
    5. diabete
    6. hypercholesterolemie
    7. antecedents_familiaux
    8. tabagisme
    9. sedentarite
   10. pression_systolique
   11. pression_diastolique
   12. frequence_cardiaque
   13. douleur_thoracique
   14. dyspnee
   15. cholesterol_total
   16. ldl
   17. hdl
   18. triglycerides
   19. glycemie
   20. crp
   21. infarctus


In [132]:
"""
2. IDENTIFICATION DES TYPES DE VARIABLES
"""
print("\n" + "=" * 80)
print("2. IDENTIFICATION DES TYPES DE VARIABLES")
print("=" * 80)

def identifier_types_variables(dataframe):
    """Identifie les variables binaires, catégorielles et numériques"""
    variables_binaires = []
    variables_categorielles = []
    variables_numeriques = []

    for col in dataframe.columns:
        if col == 'infarctus':  # Exclure la variable cible
            continue

        valeurs_uniques = dataframe[col].dropna().unique()

        # Variables binaires (0 et 1)
        if len(valeurs_uniques) == 2 and set(valeurs_uniques).issubset({0, 1, 0.0, 1.0}):
            variables_binaires.append(col)
        # Variables catégorielles (type object ou nombre limité de valeurs uniques)
        elif dataframe[col].dtype == 'object' or (len(valeurs_uniques) < 10 and dataframe[col].dtype in ['int64', 'float64']):
            variables_categorielles.append(col)
        # Variables numeriques continues
        elif dataframe[col].dtype in ['int64', 'float64', 'int32', 'float32']:
            variables_numeriques.append(col)

    return variables_binaires, variables_categorielles, variables_numeriques

variables_binaires, variables_categorielles, variables_numeriques = identifier_types_variables(df)

print(f"\nVariables binaires ({len(variables_binaires)}):")
for i, var in enumerate(variables_binaires, 1):
    count = df[var].value_counts()
    print(f"   {i:2d}. {var:30s} - 0: {count.get(0, 0):4d}, 1: {count.get(1, 0):4d}")

print(f"\nVariables categorielles ({len(variables_categorielles)}):")
for i, var in enumerate(variables_categorielles, 1):
    valeurs_uniques = sorted(df[var].dropna().unique())
    count = df[var].value_counts().sort_index()
    print(f"   {i:2d}. {var:30s} - Valeurs: {valeurs_uniques}")
    for val in valeurs_uniques:
        print(f"       Valeur {val}: {count.get(val, 0):4d} patients")

print(f"\nVariables numeriques continues ({len(variables_numeriques)}):")
for i, var in enumerate(variables_numeriques, 1):
    min_val = df[var].min()
    max_val = df[var].max()
    mean_val = df[var].mean()
    print(f"   {i:2d}. {var:30s} - Min: {min_val:8.2f}, Max: {max_val:8.2f}, Moyenne: {mean_val:8.2f}")

# Separer les features de la variable cible
if 'infarctus' in df.columns:
    X = df.drop(columns=['infarctus'])
    y = df['infarctus']
else:
    X = df.copy()
    y = None



2. IDENTIFICATION DES TYPES DE VARIABLES

Variables binaires (8):
    1. sexe                           - 0:  458, 1:  542
    2. hypertension                   - 0:  381, 1:  619
    3. diabete                        - 0:  532, 1:  468
    4. hypercholesterolemie           - 0:  598, 1:  402
    5. antecedents_familiaux          - 0:  728, 1:  272
    6. sedentarite                    - 0:  506, 1:  494
    7. douleur_thoracique             - 0:  651, 1:  349
    8. dyspnee                        - 0:  733, 1:  267

Variables categorielles (1):
    1. tabagisme                      - Valeurs: [np.int64(0), np.int64(1), np.int64(2)]
       Valeur 0:  525 patients
       Valeur 1:  249 patients
       Valeur 2:  226 patients

Variables numeriques continues (11):
    1. age                            - Min:     2.00, Max:   252.59, Moyenne:    55.32
    2. imc                            - Min:    18.00, Max:    39.55, Moyenne:    27.07
    3. pression_systolique            - Min:    93.

In [133]:
"""
3. GESTION DES OUTLIERS (VALEURS ABERRANTES)
"""
print("\n" + "=" * 80)
print("3. GESTION DES OUTLIERS - METHODE IQR")
print("=" * 80)

# >>> PARAMETRE MODIFIABLE : Facteur IQR <<<
FACTEUR_IQR = 2.5
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

print(f"\nConfiguration : FACTEUR_IQR = {FACTEUR_IQR}")

def detecter_outliers_iqr(dataframe, colonne, facteur_iqr):
    Q1 = dataframe[colonne].quantile(0.25)
    Q3 = dataframe[colonne].quantile(0.75)
    IQR = Q3 - Q1

    limite_inf = Q1 - facteur_iqr * IQR
    limite_sup = Q3 + facteur_iqr * IQR

    outliers = dataframe[(dataframe[colonne] < limite_inf) | (dataframe[colonne] > limite_sup)][colonne]

    return {
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'limite_inf': limite_inf,
        'limite_sup': limite_sup,
        'n_outliers': len(outliers),
        'pct_outliers': (len(outliers) / len(dataframe)) * 100,
        'outliers_indices': outliers.index.tolist()
    }

print("\n" + "-" * 80)
print("ANALYSE DES OUTLIERS PAR VARIABLE NUMERIQUE")
print("-" * 80)

# On analyse uniquement les variables numeriques continues (pas les binaires)
variables_a_analyser = [var for var in variables_numeriques]

print(f"\nNombre de variables a analyser : {len(variables_a_analyser)}")

# Dictionnaire pour stocker les resultats
resultats_outliers = {}

for var in variables_a_analyser:
    stats = detecter_outliers_iqr(df, var, facteur_iqr=FACTEUR_IQR)
    resultats_outliers[var] = stats

    print(f"\n{var.upper()}")
    print(f"   Q1                  : {stats['Q1']:>10.2f}")
    print(f"   Q3                  : {stats['Q3']:>10.2f}")
    print(f"   IQR                 : {stats['IQR']:>10.2f}")
    print(f"   Limite inferieure   : {stats['limite_inf']:>10.2f}")
    print(f"   Limite superieure   : {stats['limite_sup']:>10.2f}")
    print(f"   Outliers detectes   : {stats['n_outliers']:>10d} ({stats['pct_outliers']:>6.2f}%)")

    if stats['n_outliers'] > 0:
        print(f"   --> ATTENTION : Valeurs aberrantes detectees")

print("\n" + "-" * 80)

print("\n" + "-" * 80)
print("TRAITEMENT : REMPLACEMENT DES OUTLIERS PAR NaN")
print("-" * 80)

# Creer une copie pour le traitement
df_cleaned = df.copy()

# Compter le nombre total de valeurs aberrantes remplacees
total_outliers_remplaces = 0
details_remplacement = []

for var in variables_a_analyser:
    stats = resultats_outliers[var]

    if stats['n_outliers'] > 0:
        # Identifier les outliers
        condition_outliers = ((df_cleaned[var] < stats['limite_inf']) |
                             (df_cleaned[var] > stats['limite_sup']))

        n_avant = df_cleaned[var].notna().sum()

        # Remplacer les outliers par NaN
        df_cleaned.loc[condition_outliers, var] = np.nan

        n_apres = df_cleaned[var].notna().sum()
        n_remplaces = n_avant - n_apres
        total_outliers_remplaces += n_remplaces

        details_remplacement.append({
            'Variable': var,
            'Outliers remplaces': n_remplaces,
            'Pourcentage': (n_remplaces / len(df_cleaned)) * 100
        })

        print(f"\n{var:30s} : {n_remplaces:4d} valeurs remplacees ({(n_remplaces / len(df_cleaned)) * 100:5.2f}%)")

print(f"\n{'-' * 80}")
print(f"TOTAL : {total_outliers_remplaces} valeurs aberrantes remplacees par NaN")
print(f"{'-' * 80}")


3. GESTION DES OUTLIERS - METHODE IQR

Configuration : FACTEUR_IQR = 2.5

--------------------------------------------------------------------------------
ANALYSE DES OUTLIERS PAR VARIABLE NUMERIQUE
--------------------------------------------------------------------------------

Nombre de variables a analyser : 11

AGE
   Q1                  :      48.49
   Q3                  :      61.48
   IQR                 :      12.99
   Limite inferieure   :      16.02
   Limite superieure   :      93.95
   Outliers detectes   :          2 (  0.20%)
   --> ATTENTION : Valeurs aberrantes detectees

IMC
   Q1                  :      24.43
   Q3                  :      29.66
   IQR                 :       5.23
   Limite inferieure   :      11.34
   Limite superieure   :      42.75
   Outliers detectes   :          0 (  0.00%)

PRESSION_SYSTOLIQUE
   Q1                  :     130.69
   Q3                  :     148.42
   IQR                 :      17.74
   Limite inferieure   :      86.35
   Limi

In [134]:
"""
4. PRETRAITEMENT - SPLIT TRAIN/TEST
"""
print("\n" + "=" * 80)
print("4. PRETRAITEMENT - SPLIT TRAIN/TEST")
print("=" * 80)

# Mise a jour du dataframe apres traitement des outliers
df = df_cleaned.copy()

# Separation features et cible
X = df.drop(columns=['infarctus'])
y = df['infarctus']

print(f"\nDataset complet :")
print(f"   Features (X)     : {X.shape[0]} lignes x {X.shape[1]} colonnes")
print(f"   Cible (y)        : {y.shape[0]} valeurs")
print(f"   Distribution y   : {y.value_counts().to_dict()}")

# Split train/test avec stratification (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"\n" + "-" * 80)
print("SPLIT TRAIN/TEST (80/20)")
print("-" * 80)

print(f"\nEnsemble d'entrainement :")
print(f"   X_train                              : {X_train.shape[0]} lignes x {X_train.shape[1]} colonnes")
print(f"   y_train                              : {y_train.shape[0]} valeurs")
print(f"   Distribution de la variable cible    : {y_train.value_counts().to_dict()}")
print(f"   Proportion                           : {(len(X_train) / len(X)) * 100:.1f}%")

print(f"\nEnsemble de test :")
print(f"   X_test                               : {X_test.shape[0]} lignes x {X_test.shape[1]} colonnes")
print(f"   y_test                               : {y_test.shape[0]} valeurs")
print(f"   Distribution de la variable cible    : {y_test.value_counts().to_dict()}")
print(f"   Proportion                           : {(len(X_test) / len(X)) * 100:.1f}%")

# Verification de la stratification
print(f"\n" + "-" * 80)
print("VERIFICATION DE LA STRATIFICATION")
print("-" * 80)

prop_train = y_train.value_counts(normalize=True).sort_index()
prop_test = y_test.value_counts(normalize=True).sort_index()
prop_total = y.value_counts(normalize=True).sort_index()

print(f"\nProportions de la classe positive (infarctus=1) :")
print(f"   Dataset complet  : {prop_total[1]:.4f}")
print(f"   Train            : {prop_train[1]:.4f}")
print(f"   Test             : {prop_test[1]:.4f}")
print(f"   Difference       : {abs(prop_train[1] - prop_test[1]):.4f}")

if abs(prop_train[1] - prop_test[1]) < 0.01:
    print("\n   Stratification : OK - Distributions equilibrees")
else:
    print("\n   Stratification : ATTENTION - Leger desequilibre")



4. PRETRAITEMENT - SPLIT TRAIN/TEST

Dataset complet :
   Features (X)     : 1000 lignes x 20 colonnes
   Cible (y)        : 1000 valeurs
   Distribution y   : {0: 750, 1: 250}

--------------------------------------------------------------------------------
SPLIT TRAIN/TEST (80/20)
--------------------------------------------------------------------------------

Ensemble d'entrainement :
   X_train                              : 800 lignes x 20 colonnes
   y_train                              : 800 valeurs
   Distribution de la variable cible    : {0: 600, 1: 200}
   Proportion                           : 80.0%

Ensemble de test :
   X_test                               : 200 lignes x 20 colonnes
   y_test                               : 200 valeurs
   Distribution de la variable cible    : {0: 150, 1: 50}
   Proportion                           : 20.0%

--------------------------------------------------------------------------------
VERIFICATION DE LA STRATIFICATION
----------------

In [135]:
"""
5. IMPUTATION DES VALEURS MANQUANTES
"""
print("\n" + "=" * 80)
print("5. IMPUTATION DES VALEURS MANQUANTES")
print("=" * 80)

print("""\nMETHODE : Imputation par la MEDIANE""")

# Identifier les variables numeriques a imputer (dans train OU test)
variables_a_imputer = [var for var in variables_numeriques
                       if X_train[var].isnull().sum() > 0 or X_test[var].isnull().sum() > 0]

print(f"\n" + "-" * 80)
print("ANALYSE DES VALEURS MANQUANTES")
print("-" * 80)

print(f"\nEnsemble d'entrainement (X_train) :")
missing_train = X_train.isnull().sum()
missing_train = missing_train[missing_train > 0]
if len(missing_train) > 0:
    for var in missing_train.index:
        print(f"   {var:30s} : {missing_train[var]:4d} NaN ({(missing_train[var]/len(X_train))*100:5.2f}%)")
else:
    print("   Aucune valeur manquante")

print(f"\nEnsemble de test (X_test) :")
missing_test = X_test.isnull().sum()
missing_test = missing_test[missing_test > 0]
if len(missing_test) > 0:
    for var in missing_test.index:
        print(f"   {var:30s} : {missing_test[var]:4d} NaN ({(missing_test[var]/len(X_test))*100:5.2f}%)")
else:
    print("   Aucune valeur manquante")

# Imputation
if len(variables_a_imputer) > 0:
    print(f"\n" + "-" * 80)
    print("APPLICATION DE L'IMPUTATION")
    print("-" * 80)

    print(f"\nVariables a imputer : {len(variables_a_imputer)}")

    # Creer l'imputer
    imputer = SimpleImputer(strategy='median')

    # Fit sur le train uniquement
    imputer.fit(X_train[variables_a_imputer])

    # Stocker les medianes
    medianes_train = {}
    for idx, var in enumerate(variables_a_imputer):
        medianes_train[var] = imputer.statistics_[idx]

    # Transform train et test
    X_train[variables_a_imputer] = imputer.transform(X_train[variables_a_imputer])
    X_test[variables_a_imputer] = imputer.transform(X_test[variables_a_imputer])

    print(f"\nMedianes utilisees (calculees sur train) :")
    for var, mediane in medianes_train.items():
        print(f"   {var:30s} : {mediane:10.2f}")

    print(f"\n" + "-" * 80)
    print("VERIFICATION POST-IMPUTATION")
    print("-" * 80)

    print(f"\nX_train - Valeurs manquantes : {X_train.isnull().sum().sum()}")
    print(f"X_test  - Valeurs manquantes : {X_test.isnull().sum().sum()}")

    if X_train.isnull().sum().sum() == 0 and X_test.isnull().sum().sum() == 0:
        print("\nSUCCES : Toutes les valeurs manquantes ont ete imputees")
    else:
        print("\nATTENTION : Il reste des valeurs manquantes !")

else:
    print(f"\nAucune imputation necessaire")

print("\n" + "=" * 80)
print("SPLIT ET IMPUTATION TERMINES")
print("=" * 80)

# Mise a jour de la liste des variables numeriques apres pretraitement
print("\nOn met a jour la liste des variables numeriques avec les nouvelles valeurs")

# Reconstruction du dataframe complet pour re-analyser les types de variables numeriques
df_final = pd.concat([X_train, X_test]).sort_index()
df_final['infarctus'] = pd.concat([y_train, y_test]).sort_index()
_, _, variables_numeriques = identifier_types_variables(df_final)


5. IMPUTATION DES VALEURS MANQUANTES

METHODE : Imputation par la MEDIANE

--------------------------------------------------------------------------------
ANALYSE DES VALEURS MANQUANTES
--------------------------------------------------------------------------------

Ensemble d'entrainement (X_train) :
   age                            :    2 NaN ( 0.25%)
   pression_systolique            :    1 NaN ( 0.12%)
   frequence_cardiaque            :    1 NaN ( 0.12%)
   triglycerides                  :    1 NaN ( 0.12%)
   crp                            :   19 NaN ( 2.38%)

Ensemble de test (X_test) :
   Aucune valeur manquante

--------------------------------------------------------------------------------
APPLICATION DE L'IMPUTATION
--------------------------------------------------------------------------------

Variables a imputer : 5

Medianes utilisees (calculees sur train) :
   age                            :      55.16
   pression_systolique            :     140.06
   frequence_c

In [136]:
"""
6. PREPARATION DES DONNEES POUR L'APPRENTISSAGE
"""

print("\n" + "=" * 80)
print("6. PREPARATION DES DONNEES POUR L'APPRENTISSAGE")
print("=" * 80)

print("\n" + "-" * 80)
print("A. STANDARDISATION DES VARIABLES NUMERIQUES")
print("-" * 80)

print("""
METHODE : Standardisation (moyenne=0, ecart-type=1)
----------------------------------------------------
- Fit sur les donnees d'ENTRAINEMENT uniquement
- Transform sur train et test avec les parametres du train
- Evite le data leakage
""")

# Creer le scaler
scaler = StandardScaler()

print("-" * 80)
print("FIT SUR L'ENSEMBLE D'ENTRAINEMENT")
print("-" * 80)

# Variables numeriques du train
X_train_num = X_train[variables_numeriques].copy()

print(f"\nVariables numeriques a standardiser : {len(variables_numeriques)}")
print(f"Shape X_train numeriques : {X_train_num.shape}")

# Fit sur le train uniquement
scaler.fit(X_train_num)

print("\n" + "-" * 80)
print("TRANSFORM SUR TRAIN ET TEST")
print("-" * 80)

# Transform train
X_train_num_scaled = scaler.transform(X_train_num)
X_train_num_scaled_df = pd.DataFrame(
    X_train_num_scaled,
    columns=variables_numeriques,
    index=X_train_num.index
)

print(f"\nTRAIN apres standardisation :")
print(f"   Shape                        : {X_train_num_scaled_df.shape}")
print(f"   Moyenne (devrait etre 0)     : {X_train_num_scaled.mean():.6f}")
print(f"   Ecart-type (devrait etre 1)  : {X_train_num_scaled.std():.6f}")

# Transform test avec les parametres du train
X_test_num = X_test[variables_numeriques].copy()
X_test_num_scaled = scaler.transform(X_test_num)
X_test_num_scaled_df = pd.DataFrame(
    X_test_num_scaled,
    columns=variables_numeriques,
    index=X_test_num.index
)

print(f"\nTEST apres standardisation :")
print(f"   Shape                      : {X_test_num_scaled_df.shape}")
print(f"   Moyenne                    : {X_test_num_scaled.mean():.6f}")
print(f"   Ecart-type                 : {X_test_num_scaled.std():.6f}")
print(f"   (Peut differer de 0 et 1 car parametres du train appliques)")

print("\n" + "-" * 80)
print("STANDARDISATION TERMINEE")
print("-" * 80)

# Encoder les variables categorielles si elles existent (SANS standardisation)
if len(variables_categorielles) > 0:
    print("\n" + "-" * 80)
    print("B. ENCODAGE DES VARIABLES CATEGORIELLES")
    print("-" * 80)

    print(f"\nVariables categorielles a encoder : {len(variables_categorielles)}")
    print("Methode : One-Hot Encoding avec drop='first' (evite la multicollinearite)")

    # Creer l'encoder
    encoder = OneHotEncoder(sparse_output=False, drop='first')

    # Variables categorielles du train
    X_train_cat = X_train[variables_categorielles].copy()

    print(f"\nShape X_train categorielles : {X_train_cat.shape}")

    # Fit sur le train uniquement
    encoder.fit(X_train_cat)

    print("\nCategories detectees (fit sur train) :")
    for i, var in enumerate(variables_categorielles):
        print(f"   {var:30s} : {list(encoder.categories_[i])}")

    # Transform train
    X_train_cat_encoded = encoder.transform(X_train_cat)

    # Creer les noms de colonnes pour les variables encodees
    encoded_feature_names = []
    for i, var in enumerate(variables_categorielles):
        categories = encoder.categories_[i][1:]  # Exclure la premiere categorie (dropped)
        for cat in categories:
            encoded_feature_names.append(f"{var}_{cat}")

    X_train_cat_df = pd.DataFrame(
        X_train_cat_encoded,
        columns=encoded_feature_names,
        index=X_train_cat.index
    )

    # Transform test avec les parametres du train
    X_test_cat = X_test[variables_categorielles].copy()
    X_test_cat_encoded = encoder.transform(X_test_cat)

    X_test_cat_df = pd.DataFrame(
        X_test_cat_encoded,
        columns=encoded_feature_names,
        index=X_test_cat.index
    )

    print("\n" + "-" * 80)
    print("ENCODAGE TERMINE")
    print("-" * 80)

    print(f"""
Donnees encodees creees :
   - X_train_cat_df : {X_train_cat_df.shape}
   - X_test_cat_df  : {X_test_cat_df.shape}
   - Variables restent binaires (0 ou 1)
    """)

    print("\n" + "-" * 80)
    print("COMBINAISON DES VARIABLES NUMERIQUES STANDARDISEES ET CATEGORIELLES ENCODEES")
    print("-" * 80)

    # Combiner variables numeriques STANDARDISEES et categorielles encodees NON STANDARDISEES pour chaque set de données

    X_train_combined = pd.concat([X_train_num_scaled_df, X_train_cat_df], axis=1)
    X_test_combined = pd.concat([X_test_num_scaled_df, X_test_cat_df], axis=1)

    print(f"""
Donnees combinees creees :
   - X_train_combined : {X_train_combined.shape}
   - X_test_combined  : {X_test_combined.shape}
    """)

else:
    print("\n" + "-" * 80)
    print("PAS DE VARIABLES CATEGORIELLES")
    print("-" * 80)

    print("\nAucune variable categorielle a encoder")
    print("Utilisation uniquement des variables numeriques standardisees")

    X_train_combined = X_train_num_scaled_df.copy()
    X_test_combined = X_test_num_scaled_df.copy()

    print(f"\nX_train_combined : {X_train_combined.shape}")
    print(f"X_test_combined  : {X_test_combined.shape}")


print("\n" + "-" * 80)
print("C. GESTION DU DESEQUILIBRE DES CLASSES")
print("-" * 80)

# Calculer les proportions des classes
n_classe_0 = (y_train == 0).sum()
n_classe_1 = (y_train == 1).sum()
prop_classe_0 = n_classe_0 / len(y_train) * 100
prop_classe_1 = n_classe_1 / len(y_train) * 100

print("\nAnalyse du desequilibre :")
print(f"   Classe 0 (pas d'infarctus) : {n_classe_0} echantillons ({prop_classe_0:.2f}%)")
print(f"   Classe 1 (infarctus)       : {n_classe_1} echantillons ({prop_classe_1:.2f}%)")
print(f"   Ratio de desequilibre      : {n_classe_0 / n_classe_1:.2f}:1")

# Determiner si on doit utiliser class_weight='balanced'
# Condition : desequilibre entre 0/100 et 40/60 (ou 60/40)
min_prop = min(prop_classe_0, prop_classe_1)
max_prop = max(prop_classe_0, prop_classe_1)

if min_prop >= 40.0 and max_prop <= 60.0:
    # Les classes sont equilibrees (entre 40/60 et 60/40)
    class_weight = None
    print("\n" + "-" * 80)
    print("DECISION : class_weight = None")
    print("-" * 80)
    print("\nLes classes sont suffisamment equilibrees (entre 40/60 et 60/40)")
    print("Aucune ponderation necessaire")
else:
    # Les classes sont desequilibrees
    class_weight = 'balanced'
    print("\n" + "-" * 80)
    print("DECISION : class_weight = 'balanced'")
    print("-" * 80)
    print("\nLes classes sont desequilibrees (en dehors de l'intervalle 40/60 - 60/40)")
    print("Utilisation de la ponderation automatique 'balanced'")

    # Calculer les poids pour information
    class_weights_array = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train),
        y=y_train
    )

    class_weight_dict = dict(zip(np.unique(y_train), class_weights_array))

    print("\nPoids calcules pour equilibrer les classes :")
    print(f"   Classe 0 : {class_weight_dict[0]:.4f}")
    print(f"   Classe 1 : {class_weight_dict[1]:.4f}")

print("\n" + "-" * 80)
print("GESTION DU DESEQUILIBRE DES CLASSES TERMINEE")
print("-" * 80)

print("\n" + "=" * 80)
print("PREPARATION DES DONNEES TERMINEE")
print("=" * 80)

print(f"""
DATASETS FINAUX PRETS POUR LA MODELISATION :
   - X_train_combined : {X_train_combined.shape[0]} lignes x {X_train_combined.shape[1]} features
   - X_test_combined  : {X_test_combined.shape[0]} lignes x {X_test_combined.shape[1]} features
   - y_train          : {y_train.shape[0]} valeurs
   - y_test           : {y_test.shape[0]} valeurs
""")


6. PREPARATION DES DONNEES POUR L'APPRENTISSAGE

--------------------------------------------------------------------------------
A. STANDARDISATION DES VARIABLES NUMERIQUES
--------------------------------------------------------------------------------

METHODE : Standardisation (moyenne=0, ecart-type=1)
----------------------------------------------------
- Fit sur les donnees d'ENTRAINEMENT uniquement
- Transform sur train et test avec les parametres du train
- Evite le data leakage

--------------------------------------------------------------------------------
FIT SUR L'ENSEMBLE D'ENTRAINEMENT
--------------------------------------------------------------------------------

Variables numeriques a standardiser : 11
Shape X_train numeriques : (800, 11)

--------------------------------------------------------------------------------
TRANSFORM SUR TRAIN ET TEST
--------------------------------------------------------------------------------

TRAIN apres standardisation :
   Shape 

In [137]:
"""
7. IMPORTS POUR LES MODELES DE MACHINE LEARNING
"""

# Modeles supervises
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# Modeles non supervises
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import squareform
import gower

# Metriques et evaluation
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve,
    silhouette_score, davies_bouldin_score
)

print("\n" + "=" * 80)
print("7. IMPORTS POUR LES MODELES DE MACHINE LEARNING")
print("=" * 80)
print("\nTous les imports necessaires ont ete charges avec succes")


7. IMPORTS POUR LES MODELES DE MACHINE LEARNING

Tous les imports necessaires ont ete charges avec succes


In [138]:
"""
8. PREPARATION DES DIFFERENTES VERSIONS DES DONNEES
"""

print("\n" + "=" * 80)
print("8. PREPARATION DES DIFFERENTES VERSIONS DES DONNEES")
print("=" * 80)

print("""
Pour tester les differents modeles, nous preparons plusieurs versions des donnees :

1. DONNEES COMPLETES (standardisees + encodees) :
   - Pour : Regression Logistique, SVM, Random Forest, Gradient Boosting
   - Variables : Toutes (numeriques standardisees + categorielles encodees)

2. DONNEES NUMERIQUES SEULES (standardisees) :
   - Pour : PCA, K-Means (version numerique)
   - Variables : Uniquement numeriques

3. DONNEES MIXTES (non standardisees, non encodees) :
   - Pour : Distance de Gower + Clustering Hierarchique
   - Variables : Toutes (format original apres imputation)
""")

print("\n" + "-" * 80)
print("VERSION 1 : DONNEES COMPLETES (standardisees + encodees)")
print("-" * 80)

# Utiliser les variables binaires aussi
X_train_binaires = X_train[variables_binaires].copy()
X_test_binaires = X_test[variables_binaires].copy()

# Combiner numeriques standardisees + binaires + categorielles encodees
if len(variables_categorielles) > 0:
    X_train_scaled = pd.concat([X_train_num_scaled_df, X_train_binaires, X_train_cat_df], axis=1)
    X_test_scaled = pd.concat([X_test_num_scaled_df, X_test_binaires, X_test_cat_df], axis=1)
else:
    X_train_scaled = pd.concat([X_train_num_scaled_df, X_train_binaires], axis=1)
    X_test_scaled = pd.concat([X_test_num_scaled_df, X_test_binaires], axis=1)

print(f"\nX_train_scaled : {X_train_scaled.shape[0]} lignes x {X_train_scaled.shape[1]} features")
print(f"X_test_scaled  : {X_test_scaled.shape[0]} lignes x {X_test_scaled.shape[1]} features")
print(f"   - Variables numeriques standardisees : {len(variables_numeriques)}")
print(f"   - Variables binaires                 : {len(variables_binaires)}")
if len(variables_categorielles) > 0:
    print(f"   - Variables categorielles encodees   : {len(encoded_feature_names)}")

print("\n" + "-" * 80)
print("VERSION 2 : DONNEES NUMERIQUES SEULES (standardisees)")
print("-" * 80)

X_train_num_only = X_train_num_scaled_df.copy()
X_test_num_only = X_test_num_scaled_df.copy()

print(f"\nX_train_num_only : {X_train_num_only.shape[0]} lignes x {X_train_num_only.shape[1]} features")
print(f"X_test_num_only  : {X_test_num_only.shape[0]} lignes x {X_test_num_only.shape[1]} features")

print("\n" + "-" * 80)
print("VERSION 3 : DONNEES MIXTES (non standardisees, non encodees)")
print("-" * 80)

# Reconstituer le dataframe complet sans la variable cible
df_final_sans_cible = df_final.drop(columns=['infarctus'])

print(f"\ndf_final_sans_cible : {df_final_sans_cible.shape[0]} lignes x {df_final_sans_cible.shape[1]} features")
print(f"   - Variables numeriques   : {len(variables_numeriques)}")
print(f"   - Variables binaires     : {len(variables_binaires)}")
print(f"   - Variables categorielles: {len(variables_categorielles)}")

print("\n" + "=" * 80)
print("PREPARATION DES DONNEES TERMINEE")
print("=" * 80)

# Dictionnaire pour stocker les resultats des modeles
resultats_modeles = {}


8. PREPARATION DES DIFFERENTES VERSIONS DES DONNEES

Pour tester les differents modeles, nous preparons plusieurs versions des donnees :

1. DONNEES COMPLETES (standardisees + encodees) :
   - Pour : Regression Logistique, SVM, Random Forest, Gradient Boosting
   - Variables : Toutes (numeriques standardisees + categorielles encodees)

2. DONNEES NUMERIQUES SEULES (standardisees) :
   - Pour : PCA, K-Means (version numerique)
   - Variables : Uniquement numeriques

3. DONNEES MIXTES (non standardisees, non encodees) :
   - Pour : Distance de Gower + Clustering Hierarchique
   - Variables : Toutes (format original apres imputation)


--------------------------------------------------------------------------------
VERSION 1 : DONNEES COMPLETES (standardisees + encodees)
--------------------------------------------------------------------------------

X_train_scaled : 800 lignes x 21 features
X_test_scaled  : 200 lignes x 21 features
   - Variables numeriques standardisees : 11
   - Vari

In [139]:
"""
9. MODELE SUPERVISE : REGRESSION LOGISTIQUE
"""

print("\n" + "=" * 80)
print("9. MODELE SUPERVISE : REGRESSION LOGISTIQUE")
print("=" * 80)

print("""
DESCRIPTION :
   - Modele lineaire de classification binaire
   - Interprete les coefficients pour comprendre l'impact de chaque variable
   - Baseline simple et efficace

DONNEES UTILISEES :
   - X_train_scaled / X_test_scaled (toutes variables standardisees + encodees)
   - Raison : Sensible aux echelles, necessite standardisation
""")

print("\n" + "-" * 80)
print("ENTRAINEMENT DU MODELE")
print("-" * 80)

# Creer le modele
logistic_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight=class_weight
)

# Entrainer le modele
print(f"\nEntrainement sur {X_train_scaled.shape[0]} echantillons avec {X_train_scaled.shape[1]} features...")
logistic_model.fit(X_train_scaled, y_train)
print("Entrainement termine")

print("\n" + "-" * 80)
print("PREDICTIONS ET EVALUATION")
print("-" * 80)

# Predictions sur train et test
y_train_pred_lr = logistic_model.predict(X_train_scaled)
y_test_pred_lr = logistic_model.predict(X_test_scaled)

# Probabilites pour ROC-AUC
y_train_proba_lr = logistic_model.predict_proba(X_train_scaled)[:, 1]
y_test_proba_lr = logistic_model.predict_proba(X_test_scaled)[:, 1]

# Metriques sur l'ensemble d'entrainement
acc_train_lr = accuracy_score(y_train, y_train_pred_lr)
precision_train_lr = precision_score(y_train, y_train_pred_lr, zero_division=0)
recall_train_lr = recall_score(y_train, y_train_pred_lr, zero_division=0)
f1_train_lr = f1_score(y_train, y_train_pred_lr, zero_division=0)
roc_auc_train_lr = roc_auc_score(y_train, y_train_proba_lr)

# Metriques sur l'ensemble de test
acc_test_lr = accuracy_score(y_test, y_test_pred_lr)
precision_test_lr = precision_score(y_test, y_test_pred_lr, zero_division=0)
recall_test_lr = recall_score(y_test, y_test_pred_lr, zero_division=0)
f1_test_lr = f1_score(y_test, y_test_pred_lr, zero_division=0)
roc_auc_test_lr = roc_auc_score(y_test, y_test_proba_lr)

print("\nPERFORMANCES SUR L'ENSEMBLE D'ENTRAINEMENT :")
print(f"   Precision globale   : {acc_train_lr*100:.2f}%")
print(f"   Precision positive  : {precision_train_lr*100:.2f}%")
print(f"   Recall              : {recall_train_lr*100:.2f}%")
print(f"   F1-Score            : {f1_train_lr*100:.2f}%")
print(f"   ROC-AUC             : {roc_auc_train_lr*100:.2f}%")

print("\nPERFORMANCES SUR L'ENSEMBLE DE TEST :")
print(f"   Precision globale   : {acc_test_lr*100:.2f}%")
print(f"   Precision positive  : {precision_test_lr*100:.2f}%")
print(f"   Recall              : {recall_test_lr*100:.2f}%")
print(f"   F1-Score            : {f1_test_lr*100:.2f}%")
print(f"   ROC-AUC             : {roc_auc_test_lr*100:.2f}%")

print("\n" + "-" * 80)
print("MATRICE DE CONFUSION (TEST)")
print("-" * 80)

cm_lr = confusion_matrix(y_test, y_test_pred_lr)
print("\n", cm_lr)
print(f"\n   Vrais Negatifs  (TN) : {cm_lr[0, 0]}")
print(f"   Faux Positifs   (FP) : {cm_lr[0, 1]}")
print(f"   Faux Negatifs   (FN) : {cm_lr[1, 0]}")
print(f"   Vrais Positifs  (TP) : {cm_lr[1, 1]}")

print("\n" + "-" * 80)
print("RAPPORT DE CLASSIFICATION (TEST)")
print("-" * 80)
print("\n", classification_report(y_test, y_test_pred_lr, target_names=['Pas infarctus', 'Infarctus']))

print("\n" + "-" * 80)
print("IMPORTANCE DES VARIABLES")
print("-" * 80)

# Obtenir les coefficients (importance des variables)
# Pour la regression logistique, on utilise la valeur absolue des coefficients
# NORMALISATION : diviser par la somme pour etre comparable aux autres modeles
coef_abs = np.abs(logistic_model.coef_[0])
coef_normalized = coef_abs / coef_abs.sum()  # Normaliser pour que la somme = 1

feature_importance_lr = pd.DataFrame({
    'feature': X_train_scaled.columns,
    'importance': coef_normalized
}).sort_values('importance', ascending=False)

print("\n", feature_importance_lr.head(20).to_string(index=False))

# Stocker les resultats
resultats_modeles['Regression Logistique'] = {
    'modele': logistic_model,
    'accuracy_train': acc_train_lr,
    'accuracy_test': acc_test_lr,
    'precision_test': precision_test_lr,
    'recall_test': recall_test_lr,
    'f1_test': f1_test_lr,
    'roc_auc_test': roc_auc_test_lr,
    'predictions_test': y_test_pred_lr,
    'probas_test': y_test_proba_lr,
    'feature_importance': feature_importance_lr
}

print("\n" + "=" * 80)
print("REGRESSION LOGISTIQUE TERMINEE")
print("=" * 80)


9. MODELE SUPERVISE : REGRESSION LOGISTIQUE

DESCRIPTION :
   - Modele lineaire de classification binaire
   - Interprete les coefficients pour comprendre l'impact de chaque variable
   - Baseline simple et efficace

DONNEES UTILISEES :
   - X_train_scaled / X_test_scaled (toutes variables standardisees + encodees)
   - Raison : Sensible aux echelles, necessite standardisation


--------------------------------------------------------------------------------
ENTRAINEMENT DU MODELE
--------------------------------------------------------------------------------

Entrainement sur 800 echantillons avec 21 features...
Entrainement termine

--------------------------------------------------------------------------------
PREDICTIONS ET EVALUATION
--------------------------------------------------------------------------------

PERFORMANCES SUR L'ENSEMBLE D'ENTRAINEMENT :
   Precision globale   : 95.75%
   Precision positive  : 86.09%
   Recall              : 99.00%
   F1-Score            : 

In [140]:
"""
10. MODELE SUPERVISE : RANDOM FOREST CLASSIFIER
"""

print("\n" + "=" * 80)
print("10. MODELE SUPERVISE : RANDOM FOREST CLASSIFIER")
print("=" * 80)

print("""
DESCRIPTION :
   - Ensemble de arbres de decision
   - Robuste, gere bien les interactions non-lineaires
   - Peu sensible aux outliers et aux echelles

DONNEES UTILISEES :
   - X_train_scaled / X_test_scaled (toutes variables standardisees + encodees)
   - Note : Random Forest n'a pas besoin de standardisation, mais on utilise
     les memes donnees pour comparer avec les autres modeles
""")

print("\n" + "-" * 80)
print("ENTRAINEMENT DU MODELE")
print("-" * 80)

# Creer le modele
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42,
    class_weight=class_weight,
    n_jobs=-1
)

# Entrainer le modele
print(f"\nEntrainement sur {X_train_scaled.shape[0]} echantillons avec {X_train_scaled.shape[1]} features...")
rf_model.fit(X_train_scaled, y_train)
print("Entrainement termine")

print("\n" + "-" * 80)
print("PREDICTIONS ET EVALUATION")
print("-" * 80)

# Predictions sur train et test
y_train_pred_rf = rf_model.predict(X_train_scaled)
y_test_pred_rf = rf_model.predict(X_test_scaled)

# Probabilites pour ROC-AUC
y_train_proba_rf = rf_model.predict_proba(X_train_scaled)[:, 1]
y_test_proba_rf = rf_model.predict_proba(X_test_scaled)[:, 1]

# Metriques sur l'ensemble d'entrainement
acc_train_rf = accuracy_score(y_train, y_train_pred_rf)
precision_train_rf = precision_score(y_train, y_train_pred_rf, zero_division=0)
recall_train_rf = recall_score(y_train, y_train_pred_rf, zero_division=0)
f1_train_rf = f1_score(y_train, y_train_pred_rf, zero_division=0)
roc_auc_train_rf = roc_auc_score(y_train, y_train_proba_rf)

# Metriques sur l'ensemble de test
acc_test_rf = accuracy_score(y_test, y_test_pred_rf)
precision_test_rf = precision_score(y_test, y_test_pred_rf, zero_division=0)
recall_test_rf = recall_score(y_test, y_test_pred_rf, zero_division=0)
f1_test_rf = f1_score(y_test, y_test_pred_rf, zero_division=0)
roc_auc_test_rf = roc_auc_score(y_test, y_test_proba_rf)

print("\nPERFORMANCES SUR L'ENSEMBLE D'ENTRAINEMENT :")
print(f"   Precision globale   : {acc_train_rf*100:.2f}%")
print(f"   Precision positive  : {precision_train_rf*100:.2f}%")
print(f"   Recall              : {recall_train_rf*100:.2f}%")
print(f"   F1-Score            : {f1_train_rf*100:.2f}%")
print(f"   ROC-AUC             : {roc_auc_train_rf*100:.2f}%")

print("\nPERFORMANCES SUR L'ENSEMBLE DE TEST :")
print(f"   Precision globale   : {acc_test_rf*100:.2f}%")
print(f"   Precision positive  : {precision_test_rf*100:.2f}%")
print(f"   Recall              : {recall_test_rf*100:.2f}%")
print(f"   F1-Score            : {f1_test_rf*100:.2f}%")
print(f"   ROC-AUC             : {roc_auc_test_rf*100:.2f}%")

print("\n" + "-" * 80)
print("MATRICE DE CONFUSION (TEST)")
print("-" * 80)

cm_rf = confusion_matrix(y_test, y_test_pred_rf)
print("\n", cm_rf)
print(f"\n   Vrais Negatifs  (TN) : {cm_rf[0, 0]}")
print(f"   Faux Positifs   (FP) : {cm_rf[0, 1]}")
print(f"   Faux Negatifs   (FN) : {cm_rf[1, 0]}")
print(f"   Vrais Positifs  (TP) : {cm_rf[1, 1]}")

print("\n" + "-" * 80)
print("RAPPORT DE CLASSIFICATION (TEST)")
print("-" * 80)
print("\n", classification_report(y_test, y_test_pred_rf, target_names=['Pas infarctus', 'Infarctus']))

print("\n" + "-" * 80)
print("IMPORTANCE DES VARIABLES")
print("-" * 80)

# Obtenir l'importance des variables
feature_importance_rf = pd.DataFrame({
    'feature': X_train_scaled.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n", feature_importance_rf.head(20).to_string(index=False))

# Stocker les resultats
resultats_modeles['Random Forest'] = {
    'modele': rf_model,
    'accuracy_train': acc_train_rf,
    'accuracy_test': acc_test_rf,
    'precision_test': precision_test_rf,
    'recall_test': recall_test_rf,
    'f1_test': f1_test_rf,
    'roc_auc_test': roc_auc_test_rf,
    'predictions_test': y_test_pred_rf,
    'probas_test': y_test_proba_rf,
    'feature_importance': feature_importance_rf
}

print("\n" + "=" * 80)
print("RANDOM FOREST CLASSIFIER TERMINE")
print("=" * 80)


10. MODELE SUPERVISE : RANDOM FOREST CLASSIFIER

DESCRIPTION :
   - Ensemble de arbres de decision
   - Robuste, gere bien les interactions non-lineaires
   - Peu sensible aux outliers et aux echelles

DONNEES UTILISEES :
   - X_train_scaled / X_test_scaled (toutes variables standardisees + encodees)
   - Note : Random Forest n'a pas besoin de standardisation, mais on utilise
     les memes donnees pour comparer avec les autres modeles


--------------------------------------------------------------------------------
ENTRAINEMENT DU MODELE
--------------------------------------------------------------------------------

Entrainement sur 800 echantillons avec 21 features...
Entrainement termine

--------------------------------------------------------------------------------
PREDICTIONS ET EVALUATION
--------------------------------------------------------------------------------

PERFORMANCES SUR L'ENSEMBLE D'ENTRAINEMENT :
   Precision globale   : 93.62%
   Precision positive  : 81.4

In [141]:
"""
11. MODELE SUPERVISE : GRADIENT BOOSTING CLASSIFIER
"""

print("\n" + "=" * 80)
print("11. MODELE SUPERVISE : GRADIENT BOOSTING CLASSIFIER")
print("=" * 80)

print("""
DESCRIPTION :
   - Ensemble d'arbres construits sequentiellement
   - Chaque arbre corrige les erreurs des precedents
   - Souvent tres performant sur donnees tabulaires

DONNEES UTILISEES :
   - X_train_scaled / X_test_scaled (toutes variables standardisees + encodees)
""")

print("\n" + "-" * 80)
print("ENTRAINEMENT DU MODELE")
print("-" * 80)

# Creer le modele
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42
)

# Entrainer le modele
print(f"\nEntrainement sur {X_train_scaled.shape[0]} echantillons avec {X_train_scaled.shape[1]} features...")
gb_model.fit(X_train_scaled, y_train)
print("Entrainement termine")

print("\n" + "-" * 80)
print("PREDICTIONS ET EVALUATION")
print("-" * 80)

# Predictions sur train et test
y_train_pred_gb = gb_model.predict(X_train_scaled)
y_test_pred_gb = gb_model.predict(X_test_scaled)

# Probabilites pour ROC-AUC
y_train_proba_gb = gb_model.predict_proba(X_train_scaled)[:, 1]
y_test_proba_gb = gb_model.predict_proba(X_test_scaled)[:, 1]

# Metriques sur l'ensemble d'entrainement
acc_train_gb = accuracy_score(y_train, y_train_pred_gb)
precision_train_gb = precision_score(y_train, y_train_pred_gb, zero_division=0)
recall_train_gb = recall_score(y_train, y_train_pred_gb, zero_division=0)
f1_train_gb = f1_score(y_train, y_train_pred_gb, zero_division=0)
roc_auc_train_gb = roc_auc_score(y_train, y_train_proba_gb)

# Metriques sur l'ensemble de test
acc_test_gb = accuracy_score(y_test, y_test_pred_gb)
precision_test_gb = precision_score(y_test, y_test_pred_gb, zero_division=0)
recall_test_gb = recall_score(y_test, y_test_pred_gb, zero_division=0)
f1_test_gb = f1_score(y_test, y_test_pred_gb, zero_division=0)
roc_auc_test_gb = roc_auc_score(y_test, y_test_proba_gb)

print("\nPERFORMANCES SUR L'ENSEMBLE D'ENTRAINEMENT :")
print(f"   Precision globale   : {acc_train_gb*100:.2f}%")
print(f"   Precision positive  : {precision_train_gb*100:.2f}%")
print(f"   Recall              : {recall_train_gb*100:.2f}%")
print(f"   F1-Score            : {f1_train_gb*100:.2f}%")
print(f"   ROC-AUC             : {roc_auc_train_gb*100:.2f}%")

print("\nPERFORMANCES SUR L'ENSEMBLE DE TEST :")
print(f"   Precision globale   : {acc_test_gb*100:.2f}%")
print(f"   Precision positive  : {precision_test_gb*100:.2f}%")
print(f"   Recall              : {recall_test_gb*100:.2f}%")
print(f"   F1-Score            : {f1_test_gb*100:.2f}%")
print(f"   ROC-AUC             : {roc_auc_test_gb*100:.2f}%")

print("\n" + "-" * 80)
print("MATRICE DE CONFUSION (TEST)")
print("-" * 80)

cm_gb = confusion_matrix(y_test, y_test_pred_gb)
print("\n", cm_gb)
print(f"\n   Vrais Negatifs  (TN) : {cm_gb[0, 0]}")
print(f"   Faux Positifs   (FP) : {cm_gb[0, 1]}")
print(f"   Faux Negatifs   (FN) : {cm_gb[1, 0]}")
print(f"   Vrais Positifs  (TP) : {cm_gb[1, 1]}")

print("\n" + "-" * 80)
print("RAPPORT DE CLASSIFICATION (TEST)")
print("-" * 80)
print("\n", classification_report(y_test, y_test_pred_gb, target_names=['Pas infarctus', 'Infarctus']))

print("\n" + "-" * 80)
print("IMPORTANCE DES VARIABLES")
print("-" * 80)

# Obtenir l'importance des variables
feature_importance_gb = pd.DataFrame({
    'feature': X_train_scaled.columns,
    'importance': gb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n", feature_importance_gb.head(20).to_string(index=False))

# Stocker les resultats
resultats_modeles['Gradient Boosting'] = {
    'modele': gb_model,
    'accuracy_train': acc_train_gb,
    'accuracy_test': acc_test_gb,
    'precision_test': precision_test_gb,
    'recall_test': recall_test_gb,
    'f1_test': f1_test_gb,
    'roc_auc_test': roc_auc_test_gb,
    'predictions_test': y_test_pred_gb,
    'probas_test': y_test_proba_gb,
    'feature_importance': feature_importance_gb
}

print("\n" + "=" * 80)
print("GRADIENT BOOSTING CLASSIFIER TERMINE")
print("=" * 80)


11. MODELE SUPERVISE : GRADIENT BOOSTING CLASSIFIER

DESCRIPTION :
   - Ensemble d'arbres construits sequentiellement
   - Chaque arbre corrige les erreurs des precedents
   - Souvent tres performant sur donnees tabulaires

DONNEES UTILISEES :
   - X_train_scaled / X_test_scaled (toutes variables standardisees + encodees)


--------------------------------------------------------------------------------
ENTRAINEMENT DU MODELE
--------------------------------------------------------------------------------

Entrainement sur 800 echantillons avec 21 features...
Entrainement termine

--------------------------------------------------------------------------------
PREDICTIONS ET EVALUATION
--------------------------------------------------------------------------------

PERFORMANCES SUR L'ENSEMBLE D'ENTRAINEMENT :
   Precision globale   : 100.00%
   Precision positive  : 100.00%
   Recall              : 100.00%
   F1-Score            : 100.00%
   ROC-AUC             : 100.00%

PERFORMANCE

In [142]:
"""
12. MODELE SUPERVISE : SVM (SUPPORT VECTOR MACHINE)
"""

print("\n" + "=" * 80)
print("12. MODELE SUPERVISE : SVM (SUPPORT VECTOR MACHINE)")
print("=" * 80)

print("""
DESCRIPTION :
   - Trouve l'hyperplan optimal separant les classes
   - Efficace en haute dimension
   - Tres sensible aux echelles (necessite standardisation)

DONNEES UTILISEES :
   - X_train_scaled / X_test_scaled (toutes variables standardisees + encodees)
   - Raison : SVM est tres sensible aux echelles
""")

print("\n" + "-" * 80)
print("ENTRAINEMENT DU MODELE")
print("-" * 80)

# Creer le modele
svm_model = SVC(
    kernel='rbf',
    C=1.0,
    gamma='scale',
    random_state=42,
    class_weight=class_weight,
    probability=True
)

# Entrainer le modele
print(f"\nEntrainement sur {X_train_scaled.shape[0]} echantillons avec {X_train_scaled.shape[1]} features...")
svm_model.fit(X_train_scaled, y_train)
print("Entrainement termine")

print("\n" + "-" * 80)
print("PREDICTIONS ET EVALUATION")
print("-" * 80)

# Predictions sur train et test
y_train_pred_svm = svm_model.predict(X_train_scaled)
y_test_pred_svm = svm_model.predict(X_test_scaled)

# Probabilites pour ROC-AUC
y_train_proba_svm = svm_model.predict_proba(X_train_scaled)[:, 1]
y_test_proba_svm = svm_model.predict_proba(X_test_scaled)[:, 1]

# Metriques sur l'ensemble d'entrainement
acc_train_svm = accuracy_score(y_train, y_train_pred_svm)
precision_train_svm = precision_score(y_train, y_train_pred_svm, zero_division=0)
recall_train_svm = recall_score(y_train, y_train_pred_svm, zero_division=0)
f1_train_svm = f1_score(y_train, y_train_pred_svm, zero_division=0)
roc_auc_train_svm = roc_auc_score(y_train, y_train_proba_svm)

# Metriques sur l'ensemble de test
acc_test_svm = accuracy_score(y_test, y_test_pred_svm)
precision_test_svm = precision_score(y_test, y_test_pred_svm, zero_division=0)
recall_test_svm = recall_score(y_test, y_test_pred_svm, zero_division=0)
f1_test_svm = f1_score(y_test, y_test_pred_svm, zero_division=0)
roc_auc_test_svm = roc_auc_score(y_test, y_test_proba_svm)

print("\nPERFORMANCES SUR L'ENSEMBLE D'ENTRAINEMENT :")
print(f"   Precision globale   : {acc_train_svm*100:.2f}%")
print(f"   Precision positive  : {precision_train_svm*100:.2f}%")
print(f"   Recall              : {recall_train_svm*100:.2f}%")
print(f"   F1-Score            : {f1_train_svm*100:.2f}%")
print(f"   ROC-AUC             : {roc_auc_train_svm*100:.2f}%")

print("\nPERFORMANCES SUR L'ENSEMBLE DE TEST :")
print(f"   Precision globale   : {acc_test_svm*100:.2f}%")
print(f"   Precision positive  : {precision_test_svm*100:.2f}%")
print(f"   Recall              : {recall_test_svm*100:.2f}%")
print(f"   F1-Score            : {f1_test_svm*100:.2f}%")
print(f"   ROC-AUC             : {roc_auc_test_svm*100:.2f}%")

print("\n" + "-" * 80)
print("MATRICE DE CONFUSION (TEST)")
print("-" * 80)

cm_svm = confusion_matrix(y_test, y_test_pred_svm)
print("\n", cm_svm)
print(f"\n   Vrais Negatifs  (TN) : {cm_svm[0, 0]}")
print(f"   Faux Positifs   (FP) : {cm_svm[0, 1]}")
print(f"   Faux Negatifs   (FN) : {cm_svm[1, 0]}")
print(f"   Vrais Positifs  (TP) : {cm_svm[1, 1]}")

print("\n" + "-" * 80)
print("RAPPORT DE CLASSIFICATION (TEST)")
print("-" * 80)
print("\n", classification_report(y_test, y_test_pred_svm, target_names=['Pas infarctus', 'Infarctus']))

print("\n" + "-" * 80)
print("IMPORTANCE DES VARIABLES")
print("-" * 80)

# Pour SVM avec kernel RBF, on utilise la permutation importance
# Methode : mesurer l'impact de chaque variable en la permutant
from sklearn.inspection import permutation_importance

print("\nCalcul de l'importance par permutation (peut prendre quelques secondes)...")
perm_importance = permutation_importance(
    svm_model, X_test_scaled, y_test,
    n_repeats=10,
    random_state=42,
    n_jobs=-1
)

feature_importance_svm = pd.DataFrame({
    'feature': X_train_scaled.columns,
    'importance': perm_importance.importances_mean
}).sort_values('importance', ascending=False)

print("\n", feature_importance_svm.head(20).to_string(index=False))

# Stocker les resultats
resultats_modeles['SVM'] = {
    'modele': svm_model,
    'accuracy_train': acc_train_svm,
    'accuracy_test': acc_test_svm,
    'precision_test': precision_test_svm,
    'recall_test': recall_test_svm,
    'f1_test': f1_test_svm,
    'roc_auc_test': roc_auc_test_svm,
    'predictions_test': y_test_pred_svm,
    'probas_test': y_test_proba_svm,
    'feature_importance': feature_importance_svm
}

print("\n" + "=" * 80)
print("SVM TERMINE")
print("=" * 80)


12. MODELE SUPERVISE : SVM (SUPPORT VECTOR MACHINE)

DESCRIPTION :
   - Trouve l'hyperplan optimal separant les classes
   - Efficace en haute dimension
   - Tres sensible aux echelles (necessite standardisation)

DONNEES UTILISEES :
   - X_train_scaled / X_test_scaled (toutes variables standardisees + encodees)
   - Raison : SVM est tres sensible aux echelles


--------------------------------------------------------------------------------
ENTRAINEMENT DU MODELE
--------------------------------------------------------------------------------

Entrainement sur 800 echantillons avec 21 features...
Entrainement termine

--------------------------------------------------------------------------------
PREDICTIONS ET EVALUATION
--------------------------------------------------------------------------------

PERFORMANCES SUR L'ENSEMBLE D'ENTRAINEMENT :
   Precision globale   : 96.50%
   Precision positive  : 88.39%
   Recall              : 99.00%
   F1-Score            : 93.40%
   ROC-AUC

In [152]:
"""
13. MODELE NON SUPERVISE : DISTANCE DE GOWER + CLUSTERING HIERARCHIQUE
"""

print("\n" + "=" * 80)
print("13. MODELE NON SUPERVISE : DISTANCE DE GOWER + CLUSTERING HIERARCHIQUE")
print("=" * 80)

print("""
DESCRIPTION :
   - Distance de Gower : adaptee aux donnees mixtes (numeriques + categorielles + binaires)
   - Clustering hierarchique : structure hierarchique des patients
   - Methode la plus robuste pour donnees heterogenes

DONNEES UTILISEES :
   - df_final_sans_cible (toutes variables, format original apres imputation)
   - Raison : Gower gere nativement tous les types de variables
   - PAS de standardisation ni d'encodage necessaire
""")

print("\n" + "-" * 80)
print("CALCUL DE LA MATRICE DE DISTANCE DE GOWER")
print("-" * 80)

print(f"\nCalcul de la matrice de distance sur {df_final_sans_cible.shape[0]} patients...")
print("Cette operation peut prendre quelques secondes...")

# Calculer la matrice de distance de Gower
gower_matrix = gower.gower_matrix(df_final_sans_cible)

print(f"Matrice de Gower calculee : {gower_matrix.shape[0]} x {gower_matrix.shape[1]}")

print("\n" + "-" * 80)
print("CLUSTERING HIERARCHIQUE")
print("-" * 80)

# Convertir en format condensed pour linkage
gower_dist_condensed = squareform(gower_matrix, checks=False)

# Clustering hierarchique avec methode Ward
print("\nMethode : Ward (minimise la variance intra-cluster)")
linkage_matrix = linkage(gower_dist_condensed, method='ward')

print("Linkage matrix calculee")

# Tester differents nombres de clusters
print("\n" + "-" * 80)
print("EVALUATION POUR DIFFERENTS NOMBRES DE CLUSTERS")
print("-" * 80)

resultats_gower_clusters = {}

for n_clusters in [2, 3, 4, 5]:
    # Couper le dendrogramme pour obtenir n clusters
    cluster_labels = fcluster(linkage_matrix, n_clusters, criterion='maxclust')

    # Calculer les metriques
    silhouette = silhouette_score(gower_matrix, cluster_labels, metric='precomputed')
    davies_bouldin = davies_bouldin_score(gower_matrix, cluster_labels)

    # Analyser la distribution des infarctus dans chaque cluster
    df_temp = df_final.copy()
    df_temp['cluster'] = cluster_labels

    print(f"\nNOMBRE DE CLUSTERS : {n_clusters}")
    print(f"   Silhouette Score      : {silhouette*100:.2f}% (plus proche de 100% = mieux)")
    print(f"   Davies-Bouldin Score  : {davies_bouldin:.2f} (plus proche de 0 = mieux)")

    print(f"\n   Distribution des patients par cluster :")
    for cluster_id in range(1, n_clusters + 1):
        n_patients = (cluster_labels == cluster_id).sum()
        pct_patients = (n_patients / len(cluster_labels)) * 100

        # Taux d'infarctus dans ce cluster
        cluster_mask = df_temp['cluster'] == cluster_id
        n_infarctus = df_temp.loc[cluster_mask, 'infarctus'].sum()
        taux_infarctus = (n_infarctus / n_patients) * 100 if n_patients > 0 else 0

        print(f"      Cluster {cluster_id} : {n_patients:4d} patients ({pct_patients:5.1f}%) - Taux infarctus : {taux_infarctus:5.1f}%")

    resultats_gower_clusters[n_clusters] = {
        'labels': cluster_labels,
        'silhouette': silhouette,
        'davies_bouldin': davies_bouldin
    }

# Selectionner le meilleur nombre de clusters (basé sur Silhouette Score)
best_n_clusters_gower = max(resultats_gower_clusters.keys(),
                             key=lambda k: resultats_gower_clusters[k]['silhouette'])

print("\n" + "-" * 80)
print(f"MEILLEUR NOMBRE DE CLUSTERS : {best_n_clusters_gower}")
print("-" * 80)
print(f"   Basé sur le Silhouette Score le plus eleve")
print(f"   Silhouette Score : {resultats_gower_clusters[best_n_clusters_gower]['silhouette']*100:.2f}%")

# Utiliser le meilleur clustering
best_cluster_labels_gower = resultats_gower_clusters[best_n_clusters_gower]['labels']

# Ajouter les clusters au dataframe pour analyse ulterieure
df_final['cluster_gower'] = best_cluster_labels_gower

print("\n" + "-" * 80)
print("IMPORTANCE DES VARIABLES")
print("-" * 80)

print("""
Pour evaluer l'importance des variables dans le clustering,
on calcule la contribution de chaque variable a la separation des clusters.
Methode : Variance inter-cluster / Variance totale
""")

# Calculer l'importance des variables basee sur la variance inter-cluster
df_avec_clusters = df_final_sans_cible.copy()
df_avec_clusters['cluster'] = best_cluster_labels_gower

importance_gower = []

for col in df_final_sans_cible.columns:
    # Variance totale
    if df_final_sans_cible[col].dtype in ['int64', 'float64']:
        variance_totale = df_final_sans_cible[col].var()

        # Variance inter-cluster (entre les moyennes des clusters)
        moyennes_clusters = df_avec_clusters.groupby('cluster')[col].mean()
        variance_inter = moyennes_clusters.var()

        # Ratio variance inter / variance totale
        if variance_totale > 0:
            ratio = variance_inter / variance_totale
        else:
            ratio = 0.0
    else:
        # Pour les variables categorielles, on utilise l'entropie
        ratio = 0.0

    importance_gower.append({'feature': col, 'importance': ratio})

feature_importance_gower = pd.DataFrame(importance_gower).sort_values('importance', ascending=False)

print("\nTop 20 des variables les plus importantes :")
for idx, row in feature_importance_gower.head(20).iterrows():
    print(f"   {row['feature']:30s} : {row['importance']:.6f}")

# Stocker les resultats
resultats_modeles['Gower + Hierarchical'] = {
    'gower_matrix': gower_matrix,
    'linkage_matrix': linkage_matrix,
    'best_n_clusters': best_n_clusters_gower,
    'cluster_labels': best_cluster_labels_gower,
    'resultats_par_k': resultats_gower_clusters,
    'feature_importance': feature_importance_gower
}

print("\n" + "=" * 80)
print("DISTANCE DE GOWER + CLUSTERING HIERARCHIQUE TERMINE")
print("=" * 80)


13. MODELE NON SUPERVISE : DISTANCE DE GOWER + CLUSTERING HIERARCHIQUE

DESCRIPTION :
   - Distance de Gower : adaptee aux donnees mixtes (numeriques + categorielles + binaires)
   - Clustering hierarchique : structure hierarchique des patients
   - Methode la plus robuste pour donnees heterogenes

DONNEES UTILISEES :
   - df_final_sans_cible (toutes variables, format original apres imputation)
   - Raison : Gower gere nativement tous les types de variables
   - PAS de standardisation ni d'encodage necessaire


--------------------------------------------------------------------------------
CALCUL DE LA MATRICE DE DISTANCE DE GOWER
--------------------------------------------------------------------------------

Calcul de la matrice de distance sur 1000 patients...
Cette operation peut prendre quelques secondes...
Matrice de Gower calculee : 1000 x 1000

--------------------------------------------------------------------------------
CLUSTERING HIERARCHIQUE
---------------------------

In [151]:
"""
14. MODELE NON SUPERVISE : PCA (ANALYSE EN COMPOSANTES PRINCIPALES)
"""

print("\n" + "=" * 80)
print("14. MODELE NON SUPERVISE : PCA (ANALYSE EN COMPOSANTES PRINCIPALES)")
print("=" * 80)

print("""
DESCRIPTION :
   - Reduction de dimensionnalite
   - Trouve les directions de variance maximale
   - Permet la visualisation en 2D/3D

DONNEES UTILISEES :
   - df_final_sans_cible avec uniquement variables numeriques standardisees
   - Raison : PCA necessite des variables numeriques standardisees
""")

print("\n" + "-" * 80)
print("PREPARATION DES DONNEES POUR PCA")
print("-" * 80)

# Extraire uniquement les variables numeriques du dataframe complet
df_num_for_pca = df_final[variables_numeriques].copy()

# Standardiser
scaler_pca = StandardScaler()
df_num_scaled_pca = scaler_pca.fit_transform(df_num_for_pca)

print(f"\nDonnees pour PCA : {df_num_scaled_pca.shape[0]} lignes x {df_num_scaled_pca.shape[1]} variables numeriques")

print("\n" + "-" * 80)
print("APPLICATION DE LA PCA")
print("-" * 80)

# Appliquer PCA avec toutes les composantes
pca_full = PCA(random_state=42)
pca_full.fit(df_num_scaled_pca)

# Variance expliquee cumulee
variance_cumulee = np.cumsum(pca_full.explained_variance_ratio_)

print("\nVariance expliquee par les composantes principales :")
for i in range(min(10, len(pca_full.explained_variance_ratio_))):
    print(f"   PC{i+1:2d} : {pca_full.explained_variance_ratio_[i]:.4f} ({variance_cumulee[i]:.4f} cumulee)")

# Trouver le nombre de composantes pour expliquer 80% et 95% de la variance
n_comp_80 = np.argmax(variance_cumulee >= 0.80) + 1
n_comp_95 = np.argmax(variance_cumulee >= 0.95) + 1

print(f"\nNombre de composantes pour expliquer :")
print(f"   80% de la variance : {n_comp_80} composantes")
print(f"   95% de la variance : {n_comp_95} composantes")

# Appliquer PCA avec reduction a 2 composantes pour visualisation
print("\n" + "-" * 80)
print("PCA AVEC 2 COMPOSANTES (POUR VISUALISATION)")
print("-" * 80)

pca_2d = PCA(n_components=2, random_state=42)
principal_components_2d = pca_2d.fit_transform(df_num_scaled_pca)

variance_expliquee_2d = pca_2d.explained_variance_ratio_.sum()

print(f"\nVariance expliquee par PC1 et PC2 : {variance_expliquee_2d:.4f} ({variance_expliquee_2d*100:.2f}%)")
print(f"   PC1 : {pca_2d.explained_variance_ratio_[0]:.4f} ({pca_2d.explained_variance_ratio_[0]*100:.2f}%)")
print(f"   PC2 : {pca_2d.explained_variance_ratio_[1]:.4f} ({pca_2d.explained_variance_ratio_[1]*100:.2f}%)")

# Creer un dataframe avec les composantes principales
df_pca_2d = pd.DataFrame(
    principal_components_2d,
    columns=['PC1', 'PC2'],
    index=df_final.index
)

# Ajouter la variable cible pour analyse
df_pca_2d['infarctus'] = df_final['infarctus'].values

print("\n" + "-" * 80)
print("ANALYSE DES COMPOSANTES PRINCIPALES")
print("-" * 80)

# Top variables contribuant a PC1 et PC2
loadings = pd.DataFrame(
    pca_2d.components_.T,
    columns=['PC1', 'PC2'],
    index=variables_numeriques
)

print("\nTop 5 variables contribuant a PC1 :")
top_pc1 = loadings['PC1'].abs().sort_values(ascending=False).head()
for var, val in top_pc1.items():
    print(f"   {var:30s} : {loadings.loc[var, 'PC1']:7.4f}")

print("\nTop 5 variables contribuant a PC2 :")
top_pc2 = loadings['PC2'].abs().sort_values(ascending=False).head()
for var, val in top_pc2.items():
    print(f"   {var:30s} : {loadings.loc[var, 'PC2']:7.4f}")

print("\n" + "-" * 80)
print("IMPORTANCE DES VARIABLES")
print("-" * 80)

print("""
Pour la PCA, l'importance d'une variable est mesuree par sa contribution globale
aux composantes principales (moyenne des valeurs absolues des loadings).
""")

# Calculer l'importance globale de chaque variable
# Moyenne des valeurs absolues des loadings sur toutes les composantes
importance_pca = pd.DataFrame({
    'feature': variables_numeriques,
    'importance': np.abs(pca_full.components_).mean(axis=0)
}).sort_values('importance', ascending=False)

print("\nTop 20 des variables les plus importantes :")
for idx, row in importance_pca.head(20).iterrows():
    print(f"   {row['feature']:30s} : {row['importance']:.6f}")

# Stocker les resultats
resultats_modeles['PCA'] = {
    'pca_full': pca_full,
    'pca_2d': pca_2d,
    'principal_components_2d': principal_components_2d,
    'variance_expliquee_2d': variance_expliquee_2d,
    'n_comp_80': n_comp_80,
    'n_comp_95': n_comp_95,
    'loadings': loadings,
    'feature_importance': importance_pca
}

print("\n" + "=" * 80)
print("PCA TERMINEE")
print("=" * 80)


14. MODELE NON SUPERVISE : PCA (ANALYSE EN COMPOSANTES PRINCIPALES)

DESCRIPTION :
   - Reduction de dimensionnalite
   - Trouve les directions de variance maximale
   - Permet la visualisation en 2D/3D

DONNEES UTILISEES :
   - df_final_sans_cible avec uniquement variables numeriques standardisees
   - Raison : PCA necessite des variables numeriques standardisees


--------------------------------------------------------------------------------
PREPARATION DES DONNEES POUR PCA
--------------------------------------------------------------------------------

Donnees pour PCA : 1000 lignes x 11 variables numeriques

--------------------------------------------------------------------------------
APPLICATION DE LA PCA
--------------------------------------------------------------------------------

Variance expliquee par les composantes principales :
   PC 1 : 0.1681 (0.1681 cumulee)
   PC 2 : 0.1488 (0.3169 cumulee)
   PC 3 : 0.1322 (0.4491 cumulee)
   PC 4 : 0.1009 (0.5501 cumulee)
  

In [145]:
"""
15. MODELE NON SUPERVISE : K-MEANS (VERSION NUMERIQUE)
"""

print("\n" + "=" * 80)
print("15. MODELE NON SUPERVISE : K-MEANS (VERSION NUMERIQUE)")
print("=" * 80)

print("""
DESCRIPTION :
   - Clustering par partitionnement
   - Utilise la distance euclidienne
   - Necessite de specifier le nombre de clusters

DONNEES UTILISEES :
   - Variables numeriques standardisees UNIQUEMENT
   - Raison : Eviter le biais lie aux variables binaires encodees
""")

print("\n" + "-" * 80)
print("PREPARATION DES DONNEES")
print("-" * 80)

# Utiliser les memes donnees que pour PCA (variables numeriques standardisees)
X_kmeans_num = df_num_scaled_pca.copy()

print(f"\nDonnees pour K-Means : {X_kmeans_num.shape[0]} lignes x {X_kmeans_num.shape[1]} variables numeriques")

print("\n" + "-" * 80)
print("EVALUATION POUR DIFFERENTS NOMBRES DE CLUSTERS")
print("-" * 80)

resultats_kmeans_num = {}
inertias_num = []

for n_clusters in range(2, 8):
    # Appliquer K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_kmeans_num)

    # Metriques
    inertia = kmeans.inertia_
    silhouette = silhouette_score(X_kmeans_num, cluster_labels)
    davies_bouldin = davies_bouldin_score(X_kmeans_num, cluster_labels)

    inertias_num.append(inertia)

    # Analyser la distribution des infarctus dans chaque cluster
    df_temp = df_final.copy()
    df_temp['cluster'] = cluster_labels

    print(f"\nNOMBRE DE CLUSTERS : {n_clusters}")
    print(f"   Inertia               : {inertia:.2f}")
    print(f"   Silhouette Score      : {silhouette*100:.2f}%")
    print(f"   Davies-Bouldin Score  : {davies_bouldin:.2f}")

    print(f"\n   Distribution des patients par cluster :")
    for cluster_id in range(n_clusters):
        n_patients = (cluster_labels == cluster_id).sum()
        pct_patients = (n_patients / len(cluster_labels)) * 100

        # Taux d'infarctus dans ce cluster
        cluster_mask = df_temp['cluster'] == cluster_id
        n_infarctus = df_temp.loc[cluster_mask, 'infarctus'].sum()
        taux_infarctus = (n_infarctus / n_patients) * 100 if n_patients > 0 else 0

        print(f"      Cluster {cluster_id} : {n_patients:4d} patients ({pct_patients:5.1f}%) - Taux infarctus : {taux_infarctus:5.1f}%")

    resultats_kmeans_num[n_clusters] = {
        'model': kmeans,
        'labels': cluster_labels,
        'inertia': inertia,
        'silhouette': silhouette,
        'davies_bouldin': davies_bouldin
    }

# Selectionner le meilleur nombre de clusters (basé sur Silhouette Score)
best_n_clusters_kmeans_num = max(resultats_kmeans_num.keys(),
                                  key=lambda k: resultats_kmeans_num[k]['silhouette'])

print("\n" + "-" * 80)
print(f"MEILLEUR NOMBRE DE CLUSTERS : {best_n_clusters_kmeans_num}")
print("-" * 80)
print(f"   Basé sur le Silhouette Score le plus eleve")
print(f"   Silhouette Score : {resultats_kmeans_num[best_n_clusters_kmeans_num]['silhouette']*100:.2f}%")

# Utiliser le meilleur clustering
best_cluster_labels_kmeans_num = resultats_kmeans_num[best_n_clusters_kmeans_num]['labels']

# Ajouter les clusters au dataframe
df_final['cluster_kmeans_num'] = best_cluster_labels_kmeans_num

print("\n" + "-" * 80)
print("IMPORTANCE DES VARIABLES")
print("-" * 80)

print("""
Pour K-Means, l'importance d'une variable est mesuree par sa contribution
a la separation des clusters (variance inter-cluster / variance totale).
""")

# Calculer l'importance des variables
df_kmeans_num_with_clusters = pd.DataFrame(
    df_num_scaled_pca,
    columns=variables_numeriques,
    index=df_final.index
)
df_kmeans_num_with_clusters['cluster'] = best_cluster_labels_kmeans_num

importance_kmeans_num = []

for col in variables_numeriques:
    # Variance totale
    variance_totale = df_kmeans_num_with_clusters[col].var()

    # Variance inter-cluster
    moyennes_clusters = df_kmeans_num_with_clusters.groupby('cluster')[col].mean()
    variance_inter = moyennes_clusters.var()

    # Ratio
    if variance_totale > 0:
        ratio = variance_inter / variance_totale
    else:
        ratio = 0.0

    importance_kmeans_num.append({'feature': col, 'importance': ratio})

feature_importance_kmeans_num = pd.DataFrame(importance_kmeans_num).sort_values('importance', ascending=False)

print("\nTop 20 des variables les plus importantes :")
for idx, row in feature_importance_kmeans_num.head(20).iterrows():
    print(f"   {row['feature']:30s} : {row['importance']:.6f}")

# Stocker les resultats
resultats_modeles['K-Means (numerique)'] = {
    'best_model': resultats_kmeans_num[best_n_clusters_kmeans_num]['model'],
    'best_n_clusters': best_n_clusters_kmeans_num,
    'cluster_labels': best_cluster_labels_kmeans_num,
    'resultats_par_k': resultats_kmeans_num,
    'inertias': inertias_num,
    'feature_importance': feature_importance_kmeans_num
}

print("\n" + "=" * 80)
print("K-MEANS (VERSION NUMERIQUE) TERMINE")
print("=" * 80)


15. MODELE NON SUPERVISE : K-MEANS (VERSION NUMERIQUE)

DESCRIPTION :
   - Clustering par partitionnement
   - Utilise la distance euclidienne
   - Necessite de specifier le nombre de clusters

DONNEES UTILISEES :
   - Variables numeriques standardisees UNIQUEMENT
   - Raison : Eviter le biais lie aux variables binaires encodees


--------------------------------------------------------------------------------
PREPARATION DES DONNEES
--------------------------------------------------------------------------------

Donnees pour K-Means : 1000 lignes x 11 variables numeriques

--------------------------------------------------------------------------------
EVALUATION POUR DIFFERENTS NOMBRES DE CLUSTERS
--------------------------------------------------------------------------------

NOMBRE DE CLUSTERS : 2
   Inertia               : 9794.22
   Silhouette Score      : 10.19%
   Davies-Bouldin Score  : 2.79

   Distribution des patients par cluster :
      Cluster 0 :  506 patients ( 50.6%

In [146]:
"""
16. MODELE NON SUPERVISE : K-MEANS (VERSION COMPLETE)
"""

print("\n" + "=" * 80)
print("16. MODELE NON SUPERVISE : K-MEANS (VERSION COMPLETE)")
print("=" * 80)

print("""
DESCRIPTION :
   - Clustering par partitionnement
   - Utilise la distance euclidienne
   - Version avec TOUTES les variables (numeriques + binaires + categorielles encodees)

DONNEES UTILISEES :
   - Toutes variables (numeriques standardisees + binaires + categorielles encodees)
   - Raison : Comparer l'impact de l'ajout des variables binaires/categorielles
""")

print("\n" + "-" * 80)
print("PREPARATION DES DONNEES")
print("-" * 80)

# Preparer les donnees completes pour le dataframe complet
df_num_complete = df_final[variables_numeriques].copy()
scaler_complete = StandardScaler()
df_num_scaled_complete = scaler_complete.fit_transform(df_num_complete)
df_num_scaled_complete = pd.DataFrame(df_num_scaled_complete, columns=variables_numeriques, index=df_final.index)

# Ajouter les variables binaires
df_bin_complete = df_final[variables_binaires].copy()

# Encoder les variables categorielles si elles existent
if len(variables_categorielles) > 0:
    encoder_complete = OneHotEncoder(sparse_output=False, drop='first')
    df_cat_complete = df_final[variables_categorielles].copy()
    cat_encoded = encoder_complete.fit_transform(df_cat_complete)

    # Noms des colonnes encodees
    encoded_names_complete = []
    for i, var in enumerate(variables_categorielles):
        categories = encoder_complete.categories_[i][1:]
        for cat in categories:
            encoded_names_complete.append(f"{var}_{cat}")

    df_cat_encoded_complete = pd.DataFrame(cat_encoded, columns=encoded_names_complete, index=df_final.index)

    # Combiner tout
    X_kmeans_complete = pd.concat([df_num_scaled_complete, df_bin_complete, df_cat_encoded_complete], axis=1)
else:
    X_kmeans_complete = pd.concat([df_num_scaled_complete, df_bin_complete], axis=1)

print(f"\nDonnees pour K-Means : {X_kmeans_complete.shape[0]} lignes x {X_kmeans_complete.shape[1]} features")
print(f"   - Variables numeriques standardisees : {len(variables_numeriques)}")
print(f"   - Variables binaires                 : {len(variables_binaires)}")
if len(variables_categorielles) > 0:
    print(f"   - Variables categorielles encodees   : {len(encoded_names_complete)}")

print("\n" + "-" * 80)
print("EVALUATION POUR DIFFERENTS NOMBRES DE CLUSTERS")
print("-" * 80)

resultats_kmeans_complete = {}
inertias_complete = []

for n_clusters in range(2, 8):
    # Appliquer K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_kmeans_complete)

    # Metriques
    inertia = kmeans.inertia_
    silhouette = silhouette_score(X_kmeans_complete, cluster_labels)
    davies_bouldin = davies_bouldin_score(X_kmeans_complete, cluster_labels)

    inertias_complete.append(inertia)

    # Analyser la distribution des infarctus dans chaque cluster
    df_temp = df_final.copy()
    df_temp['cluster'] = cluster_labels

    print(f"\nNOMBRE DE CLUSTERS : {n_clusters}")
    print(f"   Inertia               : {inertia:.2f}")
    print(f"   Silhouette Score      : {silhouette*100:.2f}%")
    print(f"   Davies-Bouldin Score  : {davies_bouldin:.2f}")

    print(f"\n   Distribution des patients par cluster :")
    for cluster_id in range(n_clusters):
        n_patients = (cluster_labels == cluster_id).sum()
        pct_patients = (n_patients / len(cluster_labels)) * 100

        # Taux d'infarctus dans ce cluster
        cluster_mask = df_temp['cluster'] == cluster_id
        n_infarctus = df_temp.loc[cluster_mask, 'infarctus'].sum()
        taux_infarctus = (n_infarctus / n_patients) * 100 if n_patients > 0 else 0

        print(f"      Cluster {cluster_id} : {n_patients:4d} patients ({pct_patients:5.1f}%) - Taux infarctus : {taux_infarctus:5.1f}%")

    resultats_kmeans_complete[n_clusters] = {
        'model': kmeans,
        'labels': cluster_labels,
        'inertia': inertia,
        'silhouette': silhouette,
        'davies_bouldin': davies_bouldin
    }

# Selectionner le meilleur nombre de clusters (basé sur Silhouette Score)
best_n_clusters_kmeans_complete = max(resultats_kmeans_complete.keys(),
                                       key=lambda k: resultats_kmeans_complete[k]['silhouette'])

print("\n" + "-" * 80)
print(f"MEILLEUR NOMBRE DE CLUSTERS : {best_n_clusters_kmeans_complete}")
print("-" * 80)
print(f"   Basé sur le Silhouette Score le plus eleve")
print(f"   Silhouette Score : {resultats_kmeans_complete[best_n_clusters_kmeans_complete]['silhouette']*100:.2f}%")

# Utiliser le meilleur clustering
best_cluster_labels_kmeans_complete = resultats_kmeans_complete[best_n_clusters_kmeans_complete]['labels']

# Ajouter les clusters au dataframe
df_final['cluster_kmeans_complete'] = best_cluster_labels_kmeans_complete

print("\n" + "-" * 80)
print("IMPORTANCE DES VARIABLES")
print("-" * 80)

print("""
Pour K-Means, l'importance d'une variable est mesuree par sa contribution
a la separation des clusters (variance inter-cluster / variance totale).
""")

# Calculer l'importance des variables
X_kmeans_complete_with_clusters = X_kmeans_complete.copy()
X_kmeans_complete_with_clusters['cluster'] = best_cluster_labels_kmeans_complete

importance_kmeans_complete = []

for col in X_kmeans_complete.columns:
    # Variance totale
    variance_totale = X_kmeans_complete[col].var()

    # Variance inter-cluster
    moyennes_clusters = X_kmeans_complete_with_clusters.groupby('cluster')[col].mean()
    variance_inter = moyennes_clusters.var()

    # Ratio
    if variance_totale > 0:
        ratio = variance_inter / variance_totale
    else:
        ratio = 0.0

    importance_kmeans_complete.append({'feature': col, 'importance': ratio})

feature_importance_kmeans_complete = pd.DataFrame(importance_kmeans_complete).sort_values('importance', ascending=False)

print("\nTop 20 des variables les plus importantes :")
for idx, row in feature_importance_kmeans_complete.head(20).iterrows():
    print(f"   {row['feature']:30s} : {row['importance']:.6f}")

# Stocker les resultats
resultats_modeles['K-Means (complete)'] = {
    'best_model': resultats_kmeans_complete[best_n_clusters_kmeans_complete]['model'],
    'best_n_clusters': best_n_clusters_kmeans_complete,
    'cluster_labels': best_cluster_labels_kmeans_complete,
    'resultats_par_k': resultats_kmeans_complete,
    'inertias': inertias_complete,
    'feature_importance': feature_importance_kmeans_complete
}

print("\n" + "=" * 80)
print("K-MEANS (VERSION COMPLETE) TERMINE")
print("=" * 80)


16. MODELE NON SUPERVISE : K-MEANS (VERSION COMPLETE)

DESCRIPTION :
   - Clustering par partitionnement
   - Utilise la distance euclidienne
   - Version avec TOUTES les variables (numeriques + binaires + categorielles encodees)

DONNEES UTILISEES :
   - Toutes variables (numeriques standardisees + binaires + categorielles encodees)
   - Raison : Comparer l'impact de l'ajout des variables binaires/categorielles


--------------------------------------------------------------------------------
PREPARATION DES DONNEES
--------------------------------------------------------------------------------

Donnees pour K-Means : 1000 lignes x 21 features
   - Variables numeriques standardisees : 11
   - Variables binaires                 : 8
   - Variables categorielles encodees   : 2

--------------------------------------------------------------------------------
EVALUATION POUR DIFFERENTS NOMBRES DE CLUSTERS
--------------------------------------------------------------------------------

N

In [147]:
"""
17. COMPARAISON DES MODELES SUPERVISES
"""

print("\n" + "=" * 80)
print("17. COMPARAISON DES MODELES SUPERVISES")
print("=" * 80)

print("\nRecapitulatif des performances de tous les modeles supervises sur l'ensemble de TEST :")

# Creer un tableau comparatif
comparaison_modeles = []

for nom_modele in ['Regression Logistique', 'Random Forest', 'Gradient Boosting', 'SVM']:
    if nom_modele in resultats_modeles:
        res = resultats_modeles[nom_modele]
        comparaison_modeles.append({
            'Modele': nom_modele,
            'Precision globale (%)': f"{res['accuracy_test']*100:.2f}",
            'Precision positive (%)': f"{res['precision_test']*100:.2f}",
            'Recall (%)': f"{res['recall_test']*100:.2f}",
            'F1-Score (%)': f"{res['f1_test']*100:.2f}",
            'ROC-AUC (%)': f"{res['roc_auc_test']*100:.2f}"
        })

df_comparaison = pd.DataFrame(comparaison_modeles)

print("\n" + "-" * 80)
print("TABLEAU COMPARATIF")
print("-" * 80)
print("\n", df_comparaison.to_string(index=False))

# Identifier le meilleur modele pour chaque metrique (en utilisant les valeurs numeriques)
print("\n" + "-" * 80)
print("MEILLEURS MODELES PAR METRIQUE")
print("-" * 80)

meilleurs = {}
comparaison_numerique = []

for nom_modele in ['Regression Logistique', 'Random Forest', 'Gradient Boosting', 'SVM']:
    if nom_modele in resultats_modeles:
        res = resultats_modeles[nom_modele]
        comparaison_numerique.append({
            'Modele': nom_modele,
            'Precision globale': res['accuracy_test'],
            'Precision positive': res['precision_test'],
            'Recall': res['recall_test'],
            'F1-Score': res['f1_test'],
            'ROC-AUC': res['roc_auc_test']
        })

df_comparaison_num = pd.DataFrame(comparaison_numerique)

for metrique in ['Precision globale', 'Precision positive', 'Recall', 'F1-Score', 'ROC-AUC']:
    idx_max = df_comparaison_num[metrique].idxmax()
    meilleur_modele = df_comparaison_num.loc[idx_max, 'Modele']
    meilleure_valeur = df_comparaison_num.loc[idx_max, metrique]
    meilleurs[metrique] = (meilleur_modele, meilleure_valeur)
    print(f"\n{metrique:20s} : {meilleur_modele:25s} ({meilleure_valeur*100:.2f}%)")

# Modele recommande (basé sur F1-Score qui equilibre precision et recall)
modele_recommande = meilleurs['F1-Score'][0]
f1_recommande = meilleurs['F1-Score'][1]

print("\n" + "=" * 80)
print(f"MODELE RECOMMANDE : {modele_recommande}")
print("=" * 80)
print(f"\nCritere : F1-Score (equilibre entre precision positive et recall)")
print(f"F1-Score : {f1_recommande*100:.2f}%")
print(f"\nCe modele offre le meilleur compromis pour predire le risque d'infarctus")

print("\n" + "-" * 80)
print("ANALYSE DU SURAPPRENTISSAGE (OVERFITTING)")
print("-" * 80)

for nom_modele in ['Regression Logistique', 'Random Forest', 'Gradient Boosting', 'SVM']:
    if nom_modele in resultats_modeles:
        res = resultats_modeles[nom_modele]
        acc_train = res['accuracy_train']
        acc_test = res['accuracy_test']
        diff = acc_train - acc_test

        print(f"\n{nom_modele:25s}")
        print(f"   Precision globale Train : {acc_train*100:.2f}%")
        print(f"   Precision globale Test  : {acc_test*100:.2f}%")
        print(f"   Difference              : {diff*100:.2f}%", end="")

        if diff < 0.05:
            print(" - Bon equilibre (pas de surapprentissage)")
        elif diff < 0.10:
            print(" - Leger surapprentissage")
        else:
            print(" - Surapprentissage important")

print("\n" + "-" * 80)
print("COMPARAISON DE L'IMPORTANCE DES VARIABLES")
print("-" * 80)

print("""
Analyse des variables les plus importantes selon chaque modele.
Cela permet d'identifier les facteurs de risque principaux d'infarctus.
""")

for nom_modele in ['Regression Logistique', 'Random Forest', 'Gradient Boosting', 'SVM']:
    if nom_modele in resultats_modeles and 'feature_importance' in resultats_modeles[nom_modele]:
        print(f"\n{nom_modele.upper()}")
        print("-" * 80)

        feature_imp = resultats_modeles[nom_modele]['feature_importance']
        print("\nTop 20 des variables les plus importantes :")

        for idx, row in feature_imp.head(20).iterrows():
            print(f"   {row['feature']:30s} : {row['importance']:.6f}")

print("\n" + "-" * 80)
print("VARIABLES COMMUNES DANS LE TOP 20 DE TOUS LES MODELES")
print("-" * 80)

# Identifier les variables qui apparaissent dans le top 20 de tous les modeles
top20_par_modele = {}
for nom_modele in ['Regression Logistique', 'Random Forest', 'Gradient Boosting', 'SVM']:
    if nom_modele in resultats_modeles and 'feature_importance' in resultats_modeles[nom_modele]:
        feature_imp = resultats_modeles[nom_modele]['feature_importance']
        top20_par_modele[nom_modele] = set(feature_imp.head(20)['feature'].tolist())

# Trouver l'intersection
if len(top20_par_modele) > 0:
    variables_communes = set.intersection(*top20_par_modele.values())

    if len(variables_communes) > 0:
        print(f"\nVariables presentes dans le TOP 20 de tous les modeles ({len(variables_communes)}) :")
        print("\nCe sont les facteurs de risque les PLUS CRITIQUES identifies par tous les modeles.")
        print("Classement par importance moyenne (sur les 4 modeles) :")
        print("-" * 80)

        # Calculer l'importance moyenne de chaque variable commune
        importance_moyenne = {}
        for var in variables_communes:
            importances = []
            for nom_modele in ['Regression Logistique', 'Random Forest', 'Gradient Boosting', 'SVM']:
                if nom_modele in resultats_modeles and 'feature_importance' in resultats_modeles[nom_modele]:
                    feature_imp = resultats_modeles[nom_modele]['feature_importance']
                    importance_var = feature_imp[feature_imp['feature'] == var]['importance'].values
                    if len(importance_var) > 0:
                        importances.append(importance_var[0])

            if len(importances) > 0:
                importance_moyenne[var] = np.mean(importances)

        # Trier par importance moyenne decroissante
        variables_triees = sorted(importance_moyenne.items(), key=lambda x: x[1], reverse=True)

        print(f"\n{'Rang':>4s}  {'Variable':30s}  {'Importance moyenne':>18s}")
        print("-" * 80)
        for rang, (var, imp_moy) in enumerate(variables_triees, 1):
            print(f"{rang:4d}. {var:30s} : {imp_moy:.6f}")
    else:
        print("\nAucune variable commune dans le TOP 20 de tous les modeles.")
        print("\nCela suggere que les modeles utilisent des strategies differentes")
        print("pour predire le risque d'infarctus.")

print("\n" + "=" * 80)
print("COMPARAISON DES MODELES SUPERVISES TERMINEE")
print("=" * 80)


17. COMPARAISON DES MODELES SUPERVISES

Recapitulatif des performances de tous les modeles supervises sur l'ensemble de TEST :

--------------------------------------------------------------------------------
TABLEAU COMPARATIF
--------------------------------------------------------------------------------

                Modele Precision globale (%) Precision positive (%) Recall (%) F1-Score (%) ROC-AUC (%)
Regression Logistique                 95.50                  84.75     100.00        91.74       99.80
        Random Forest                 89.50                  76.36      84.00        80.00       96.25
    Gradient Boosting                 92.50                  88.89      80.00        84.21       97.33
                  SVM                 91.50                  77.05      94.00        84.68       97.53

--------------------------------------------------------------------------------
MEILLEURS MODELES PAR METRIQUE
------------------------------------------------------------

In [148]:
"""
18. COMPARAISON DES MODELES NON SUPERVISES
"""

print("\n" + "=" * 80)
print("18. COMPARAISON DES MODELES NON SUPERVISES")
print("=" * 80)

print("\nRecapitulatif des resultats des modeles de clustering :")

print("\n" + "-" * 80)
print("1. DISTANCE DE GOWER + CLUSTERING HIERARCHIQUE")
print("-" * 80)

if 'Gower + Hierarchical' in resultats_modeles:
    res_gower = resultats_modeles['Gower + Hierarchical']
    best_k = res_gower['best_n_clusters']
    res_k = res_gower['resultats_par_k'][best_k]

    print(f"\nNombre de clusters optimal : {best_k}")
    print(f"Silhouette Score           : {res_k['silhouette']*100:.2f}%")
    print(f"Davies-Bouldin Score       : {res_k['davies_bouldin']:.2f}")

    # Taux d'infarctus par cluster
    print(f"\nTaux d'infarctus par cluster :")
    for cluster_id in range(1, best_k + 1):
        cluster_mask = df_final['cluster_gower'] == cluster_id
        n_patients = cluster_mask.sum()
        n_infarctus = df_final.loc[cluster_mask, 'infarctus'].sum()
        taux = (n_infarctus / n_patients * 100) if n_patients > 0 else 0
        print(f"   Cluster {cluster_id} : {n_patients:4d} patients - {taux:5.1f}% infarctus")

print("\n" + "-" * 80)
print("2. K-MEANS (VERSION NUMERIQUE)")
print("-" * 80)

if 'K-Means (numerique)' in resultats_modeles:
    res_kmeans_num = resultats_modeles['K-Means (numerique)']
    best_k = res_kmeans_num['best_n_clusters']
    res_k = res_kmeans_num['resultats_par_k'][best_k]

    print(f"\nNombre de clusters optimal : {best_k}")
    print(f"Silhouette Score           : {res_k['silhouette']*100:.2f}%")
    print(f"Davies-Bouldin Score       : {res_k['davies_bouldin']:.2f}")
    print(f"Inertia                    : {res_k['inertia']:.2f}")

    # Taux d'infarctus par cluster
    print(f"\nTaux d'infarctus par cluster :")
    for cluster_id in range(best_k):
        cluster_mask = df_final['cluster_kmeans_num'] == cluster_id
        n_patients = cluster_mask.sum()
        n_infarctus = df_final.loc[cluster_mask, 'infarctus'].sum()
        taux = (n_infarctus / n_patients * 100) if n_patients > 0 else 0
        print(f"   Cluster {cluster_id} : {n_patients:4d} patients - {taux:5.1f}% infarctus")

print("\n" + "-" * 80)
print("3. K-MEANS (VERSION COMPLETE)")
print("-" * 80)

if 'K-Means (complete)' in resultats_modeles:
    res_kmeans_comp = resultats_modeles['K-Means (complete)']
    best_k = res_kmeans_comp['best_n_clusters']
    res_k = res_kmeans_comp['resultats_par_k'][best_k]

    print(f"\nNombre de clusters optimal : {best_k}")
    print(f"Silhouette Score           : {res_k['silhouette']*100:.2f}%")
    print(f"Davies-Bouldin Score       : {res_k['davies_bouldin']:.2f}")
    print(f"Inertia                    : {res_k['inertia']:.2f}")

    # Taux d'infarctus par cluster
    print(f"\nTaux d'infarctus par cluster :")
    for cluster_id in range(best_k):
        cluster_mask = df_final['cluster_kmeans_complete'] == cluster_id
        n_patients = cluster_mask.sum()
        n_infarctus = df_final.loc[cluster_mask, 'infarctus'].sum()
        taux = (n_infarctus / n_patients * 100) if n_patients > 0 else 0
        print(f"   Cluster {cluster_id} : {n_patients:4d} patients - {taux:5.1f}% infarctus")

print("\n" + "-" * 80)
print("4. PCA (VARIANCE EXPLIQUEE)")
print("-" * 80)

if 'PCA' in resultats_modeles:
    res_pca = resultats_modeles['PCA']
    print(f"\nVariance expliquee par PC1 et PC2 : {res_pca['variance_expliquee_2d']:.4f} ({res_pca['variance_expliquee_2d']*100:.2f}%)")
    print(f"Nombre de composantes pour 80% de variance : {res_pca['n_comp_80']}")
    print(f"Nombre de composantes pour 95% de variance : {res_pca['n_comp_95']}")

print("\n" + "-" * 80)
print("COMPARAISON DES METHODES DE CLUSTERING")
print("-" * 80)

# Comparer les Silhouette Scores
comparaison_clustering = []

if 'Gower + Hierarchical' in resultats_modeles:
    res = resultats_modeles['Gower + Hierarchical']
    k = res['best_n_clusters']
    comparaison_clustering.append({
        'Methode': 'Gower + Hierarchical',
        'N_clusters': k,
        'Silhouette (%)': f"{res['resultats_par_k'][k]['silhouette']*100:.2f}",
        'Davies-Bouldin': f"{res['resultats_par_k'][k]['davies_bouldin']:.2f}"
    })

if 'K-Means (numerique)' in resultats_modeles:
    res = resultats_modeles['K-Means (numerique)']
    k = res['best_n_clusters']
    comparaison_clustering.append({
        'Methode': 'K-Means (numerique)',
        'N_clusters': k,
        'Silhouette (%)': f"{res['resultats_par_k'][k]['silhouette']*100:.2f}",
        'Davies-Bouldin': f"{res['resultats_par_k'][k]['davies_bouldin']:.2f}"
    })

if 'K-Means (complete)' in resultats_modeles:
    res = resultats_modeles['K-Means (complete)']
    k = res['best_n_clusters']
    comparaison_clustering.append({
        'Methode': 'K-Means (complete)',
        'N_clusters': k,
        'Silhouette (%)': f"{res['resultats_par_k'][k]['silhouette']*100:.2f}",
        'Davies-Bouldin': f"{res['resultats_par_k'][k]['davies_bouldin']:.2f}"
    })

df_clustering = pd.DataFrame(comparaison_clustering)
print("\n", df_clustering.to_string(index=False))

print("\n" + "-" * 80)
print("COMPARAISON DE L'IMPORTANCE DES VARIABLES (MODELES NON SUPERVISES)")
print("-" * 80)

print("""
Analyse des variables les plus importantes selon chaque modele de clustering.
Cela permet d'identifier les caracteristiques qui differencient le mieux les groupes de patients.
""")

for nom_modele in ['Gower + Hierarchical', 'K-Means (numerique)', 'K-Means (complete)', 'PCA']:
    if nom_modele in resultats_modeles and 'feature_importance' in resultats_modeles[nom_modele]:
        print(f"\n{nom_modele.upper()}")
        print("-" * 80)

        feature_imp = resultats_modeles[nom_modele]['feature_importance']
        print("\nTop 20 des variables les plus importantes :")

        for idx, row in feature_imp.head(20).iterrows():
            print(f"   {row['feature']:30s} : {row['importance']:.6f}")

print("\n" + "-" * 80)
print("VARIABLES COMMUNES DANS LE TOP 20 DE TOUS LES MODELES NON SUPERVISES")
print("-" * 80)

# Identifier les variables qui apparaissent dans le top 20 de tous les modeles
top20_par_modele_ns = {}
for nom_modele in ['Gower + Hierarchical', 'K-Means (numerique)', 'K-Means (complete)', 'PCA']:
    if nom_modele in resultats_modeles and 'feature_importance' in resultats_modeles[nom_modele]:
        feature_imp = resultats_modeles[nom_modele]['feature_importance']
        top20_par_modele_ns[nom_modele] = set(feature_imp.head(20)['feature'].tolist())

# Trouver l'intersection
if len(top20_par_modele_ns) > 0:
    variables_communes_ns = set.intersection(*top20_par_modele_ns.values())

    if len(variables_communes_ns) > 0:
        print(f"\nVariables presentes dans le TOP 20 de tous les modeles non supervises ({len(variables_communes_ns)}) :")
        print("\nCe sont les caracteristiques qui differencient le mieux les groupes de patients.")
        print("Classement par importance moyenne NORMALISEE (sur les 4 modeles) :")
        print("-" * 80)

        # ETAPE 1 : Normaliser les importances de chaque modele (entre 0 et 1)
        importances_normalisees_ns = {}

        for nom_modele in ['Gower + Hierarchical', 'K-Means (numerique)', 'K-Means (complete)', 'PCA']:
            if nom_modele in resultats_modeles and 'feature_importance' in resultats_modeles[nom_modele]:
                feature_imp = resultats_modeles[nom_modele]['feature_importance'].copy()

                # Normaliser : (valeur - min) / (max - min)
                min_imp = feature_imp['importance'].min()
                max_imp = feature_imp['importance'].max()

                if max_imp > min_imp:
                    feature_imp['importance_norm'] = (feature_imp['importance'] - min_imp) / (max_imp - min_imp)
                else:
                    feature_imp['importance_norm'] = 0.0

                importances_normalisees_ns[nom_modele] = feature_imp

        # ETAPE 2 : Calculer l'importance moyenne normalisee pour chaque variable commune
        importance_moyenne_ns = {}
        for var in variables_communes_ns:
            importances_norm = []
            for nom_modele in ['Gower + Hierarchical', 'K-Means (numerique)', 'K-Means (complete)', 'PCA']:
                if nom_modele in importances_normalisees_ns:
                    feature_imp = importances_normalisees_ns[nom_modele]
                    importance_var = feature_imp[feature_imp['feature'] == var]['importance_norm'].values
                    if len(importance_var) > 0:
                        importances_norm.append(importance_var[0])

            if len(importances_norm) > 0:
                importance_moyenne_ns[var] = np.mean(importances_norm)

        # Trier par importance moyenne decroissante
        variables_triees_ns = sorted(importance_moyenne_ns.items(), key=lambda x: x[1], reverse=True)

        print(f"\n{'Rang':>4s}  {'Variable':30s}  {'Importance moyenne':>18s}")
        print("-" * 80)
        for rang, (var, imp_moy) in enumerate(variables_triees_ns, 1):
            print(f"{rang:4d}. {var:30s} : {imp_moy:.6f}")

        print("\n" + "-" * 80)
        print("NOTE : Importances normalisees entre 0 et 1 pour chaque modele avant calcul de la moyenne")
        print("-" * 80)
    else:
        print("\nAucune variable commune dans le TOP 20 de tous les modeles non supervises.")
        print("\nCela suggere que les modeles utilisent des strategies differentes")
        print("pour regrouper les patients.")

print("\n" + "=" * 80)
print("COMPARAISON DES MODELES NON SUPERVISES TERMINEE")
print("=" * 80)


18. COMPARAISON DES MODELES NON SUPERVISES

Recapitulatif des resultats des modeles de clustering :

--------------------------------------------------------------------------------
1. DISTANCE DE GOWER + CLUSTERING HIERARCHIQUE
--------------------------------------------------------------------------------

Nombre de clusters optimal : 2
Silhouette Score           : 14.11%
Davies-Bouldin Score       : 2.51

Taux d'infarctus par cluster :
   Cluster 1 :  337 patients -   3.9% infarctus
   Cluster 2 :  663 patients -  35.7% infarctus

--------------------------------------------------------------------------------
2. K-MEANS (VERSION NUMERIQUE)
--------------------------------------------------------------------------------

Nombre de clusters optimal : 2
Silhouette Score           : 10.19%
Davies-Bouldin Score       : 2.79
Inertia                    : 9794.22

Taux d'infarctus par cluster :
   Cluster 0 :  506 patients -  14.2% infarctus
   Cluster 1 :  494 patients -  36.0% infarctu